In [1]:
# Import library
import re
import sys
import os
import shutil
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import fasttext
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr

In [18]:
def relatedness(model):
    # Baca file wordsim_relatedness_goldstandard untuk evaluasi word embedding dengan teknik relatedness cbow
    dir_path_wordsim_relatedness_goldstandard = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Relatedness/wordsim_relatedness_goldstandard.txt'
    list_model_similarity = []

    list_human_score = []
    with open(dir_path_wordsim_relatedness_goldstandard , 'r') as file:
        content = file.read()
        lines = content.splitlines()
        for line in lines:
            item_in_line = line.split("\t")
            word_1 = item_in_line[0].lower()
            word_2 = item_in_line[1].lower()
            human_score = float(item_in_line[2])
            try:
                wv_1 = []
                wv_2 = []
                wv_1.append(model.get_word_vector(word_1))
                wv_2.append(model.get_word_vector(word_2))
                similarity = cosine_similarity(wv_1, wv_2)[0][0]
                list_model_similarity.append(similarity)
                list_human_score.append(human_score)
            except:
                continue
                #print("word_1", word_1)
                #print("word_2", word_2)

    rho_model, p_model = spearmanr(list_model_similarity, list_human_score)
    return rho_model


In [8]:
def word_analogy(model):
    # Baca file word_analogy untuk evaluasi word embedding dengan teknik word analogy (menggunakan library)

    dir_path_word_analogy = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Word_analogy/Word_Analogy.txt'

    true_predicted = 0
    false_predicted = 0



    with open(dir_path_word_analogy , 'r') as file:
        content = file.read()
        lines = content.splitlines()
        for line in lines:
            item_in_line = line.split(" ")
            if len(item_in_line) == 4: # untuk buang baris yang merupakan keterangan
                word_1 = item_in_line[0].lower()
                word_2 = item_in_line[1].lower()
                word_3 = item_in_line[2].lower()
                word_4 = item_in_line[3].lower()
                word_predicted_curr = ""
                try:
                    list_of_result_analogy = model.get_analogies(word_1, word_2, word_3)
                    word_predicted_curr = list_of_result_analogy[0][1] 
                    if word_predicted_curr == word_4:
                        true_predicted += 1
                    else:
                        false_predicted += 1

                except:
                    continue

    print("true_predicted", true_predicted, true_predicted / (true_predicted+false_predicted))
    print("false_predicted", false_predicted, false_predicted / (true_predicted+false_predicted))         

In [2]:
# Ambil jumlah paragraf dari file asli per dokumen
directory_path_real = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataPreprocessing/DataTahap7(DataCleaning)'

files_real = os.listdir(directory_path_real)
all_text_real = ""

# Ambil daftar nama file
for file in files_real:
    if '.DS_Store' not in file:
        list_files_real.append(file)
        
all_text_real = ""
for file in list_files_real:
    with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataPreprocessing/DataTahap7(DataCleaning)/' + file, 'r') as fileNow:
        content = fileNow.read()
        all_text_real += content


In [3]:
# Ambil jumlah paragraf dari file parafrasa per dokumen
directory_path_paraphrased = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataPreprocessing/DataTahap9(ParaphrasedPaper)'

files_paraphrased = os.listdir(directory_path_paraphrased)
all_text_paraphrased = ""

# Ambil daftar nama file
for file in files_paraphrased:
    if '.DS_Store' not in file:
        list_files_paraphrased.append(file)
        
all_text_paraphrased = ""
for file in list_files_paraphrased:
    with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataPreprocessing/DataTahap9(ParaphrasedPaper)/' + file, 'r') as fileNow:
        content = fileNow.read()
        all_text_paraphrased += content
        


In [4]:
# Melakukan lowercase lagi
all_text_real = all_text_real.lower()
all_text_paraphrased = all_text_paraphrased.lower()

In [5]:
# Menggabungkan teks asli dan parafrasa
all_text = all_text_real + '\n' + all_text_paraphrased

In [6]:
# Hanya mengambil huruf dan titik
all_text = re.sub(r'[^a-zA-Z.]', ' ', all_text)
all_text = re.sub(r'\s+', ' ', all_text)

In [7]:
# Buat file sementara
with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', 'w') as file:
    file.write(all_text)

In [28]:
# Membuat model word embedding fasttext menggunakan CBOW
model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='cbow', lr=0.01, dim=20, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0
Progress: 100.0% words/sec/thread:  131981 lr:  0.000000 avg.loss:  1.639794 ETA:   0h 0m 0s  0.3% words/sec/thread:  215333 lr:  0.009972 avg.loss:  3.069705 ETA:   0h 4m20s  0.5% words/sec/thread:  234088 lr:  0.009949 avg.loss:  2.746417 ETA:   0h 3m59s  4.6% words/sec/thread:  215173 lr:  0.009535 avg.loss:  2.222255 ETA:   0h 4m 9s 10.3% words/sec/thread:  203078 lr:  0.008965 avg.loss:  2.058223 ETA:   0h 4m 8s 11.2% words/sec/thread:  196004 lr:  0.008877 avg.loss:  2.049758 ETA:   0h 4m15s words/sec/thread:  175901 lr:  0.008657 avg.loss:  2.030978 ETA:   0h 4m37s 13.8% words/sec/thread:  173840 lr:  0.008623 avg.loss:  2.028647 ETA:   0h 4m39s 14.4% words/sec/thread:  170374 lr:  0.008557 avg.loss:  2.024665 ETA:   0h 4m43s lr:  0.008479 avg.loss:  2.020470 ETA:   0h 4m48s 15.9% words/sec/thread:  162876 lr:  0.008414 avg.loss:  2.016406 ETA:   0h 4m51s 17.0% words/sec/thread:  159214 lr:  0.008296 avg.loss:  2.008107 E

In [56]:
# save model
model_cbow.save_model("model_cbow_fasttext_1.bin")

In [10]:
# load model
model_cbow = fasttext.load_model("model_cbow_fasttext_1.bin")

In [37]:
# Cek jumlah kata dari data evaluasi yang ada dan tidak ada pada model word embedding word2vec cbow 1

dir_path_wordsim_relatedness_goldstandard = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Relatedness/wordsim_relatedness_goldstandard.txt'
list_words_exist = []
list_words_not_exist = []


with open(dir_path_wordsim_relatedness_goldstandard , 'r') as file:
    content = file.read()
    lines = content.splitlines()
    for line in lines:
        item_in_line = line.split("\t")
        word_1 = item_in_line[0].lower()
        word_2 = item_in_line[1].lower()
        
        # Cek kata pertama
        try:
            word_vector = model_cbow.get_word_vector(word_1)
            if word_1 not in list_words_exist:
                list_words_exist.append(word_1)
        except:
            if word_1 not in list_words_not_exist:
                list_words_not_exist.append(word_1)
        
        # Cek kata kedua
        try:
            word_vector = model_cbow.get_word_vector(word_2)
            if word_2 not in list_words_exist:
                list_words_exist.append(word_2)
        except:
            if word_2 not in list_words_not_exist:
                list_words_not_exist.append(word_2)

print("persentase kata ada", len(list_words_exist) / (len(list_words_exist) + len(list_words_not_exist)) )
print("persentase kata tidak ada", len(list_words_not_exist) / (len(list_words_exist) + len(list_words_not_exist)))

print(list_words_exist)
print(list_words_not_exist)

persentase kata ada 1.0
persentase kata tidak ada 0.0
['computer', 'keyboard', 'jerusalem', 'israel', 'planet', 'galaxy', 'canyon', 'landscape', 'opec', 'country', 'day', 'summer', 'dawn', 'citizen', 'people', 'environment', 'ecology', 'maradona', 'football', 'oil', 'money', 'bank', 'software', 'law', 'lawyer', 'weather', 'forecast', 'network', 'hardware', 'nature', 'fbi', 'investigation', 'wealth', 'psychology', 'freud', 'news', 'report', 'war', 'troops', 'physics', 'proton', 'stock', 'market', 'constellation', 'credit', 'card', 'hotel', 'reservation', 'closet', 'clothes', 'soap', 'opera', 'astronomer', 'space', 'movie', 'theater', 'treatment', 'recovery', 'baby', 'mother', 'deposit', 'television', 'film', 'mind', 'game', 'team', 'admission', 'ticket', 'palestinian', 'arafat', 'terror', 'boxing', 'round', 'internet', 'property', 'tennis', 'racket', 'telephone', 'communication', 'currency', 'cognition', 'seafood', 'sea', 'book', 'paper', 'library', 'depression', 'fighting', 'defeating'

In [47]:
# Baca file wordsim_relatedness_goldstandard untuk evaluasi word embedding dengan teknik relatedness cbow
dir_path_wordsim_relatedness_goldstandard = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Relatedness/wordsim_relatedness_goldstandard.txt'
list_model_similarity = []

list_human_score = []
with open(dir_path_wordsim_relatedness_goldstandard , 'r') as file:
    content = file.read()
    lines = content.splitlines()
    for line in lines:
        item_in_line = line.split("\t")
        word_1 = item_in_line[0].lower()
        word_2 = item_in_line[1].lower()
        human_score = float(item_in_line[2])
        try:
            wv_1 = []
            wv_2 = []
            wv_1.append(model_cbow.get_word_vector(word_1))
            wv_2.append(model_cbow.get_word_vector(word_2))
            similarity = cosine_similarity(wv_1, wv_2)[0][0]
            list_model_similarity.append(similarity)
            list_human_score.append(human_score)
        except:
            continue
            #print("word_1", word_1)
            #print("word_2", word_2)

rho_model, p_model = spearmanr(list_model_similarity, list_human_score)


In [48]:
# Nilai relatedness model cbow 1
print(rho_model)

0.12742615864701087


In [11]:
# Nilai word analogy model cbow 1
word_analogy(model_cbow)

true_predicted 60 0.0030699959066721244
false_predicted 19484 0.9969300040933279


In [10]:
# Membuat model word embedding fasttext menggunakan skip gram
model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='skipgram', lr=0.01, dim=20, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0
Progress: 100.0% words/sec/thread:   78230 lr:  0.000000 avg.loss:  0.589224 ETA:   0h 0m 0s 2.437059 ETA:   0h 9m31s  0.7% words/sec/thread:  101293 lr:  0.009928 avg.loss:  2.125696 ETA:   0h 9m12s  1.5% words/sec/thread:   99955 lr:  0.009855 avg.loss:  2.080027 ETA:   0h 9m15s  2.8% words/sec/thread:   94411 lr:  0.009719 avg.loss:  2.062138 ETA:   0h 9m40s  4.0% words/sec/thread:   91781 lr:  0.009599 avg.loss:  2.053148 ETA:   0h 9m49s  5.1% words/sec/thread:   89395 lr:  0.009485 avg.loss:  2.060578 ETA:   0h 9m57s6.0% words/sec/thread:   89261 lr:  0.009405 avg.loss:  2.065666 ETA:   0h 9m53s  6.4% words/sec/thread:   87979 lr:  0.009364 avg.loss:  2.067463 ETA:   0h 9m59s  6.7% words/sec/thread:   87921 lr:  0.009330 avg.loss:  2.069823 ETA:   0h 9m57s  7.1% words/sec/thread:   87919 lr:  0.009288 avg.loss:  2.071206 ETA:   0h 9m55s  9.3% words/sec/thread:   87502 lr:  0.009069 avg.loss:  2.049494 ETA:   0h 9m44s  88079

In [15]:
# save model
model_sg.save_model("model_sg_fasttext_1.bin")

In [12]:
# load model
model_sg = fasttext.load_model("model_sg_fasttext_1.bin")

In [12]:
# Cek jumlah kata dari data evaluasi yang ada dan tidak ada pada model word embedding word2vec cbow 1

dir_path_wordsim_relatedness_goldstandard = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Relatedness/wordsim_relatedness_goldstandard.txt'
list_words_exist = []
list_words_not_exist = []


with open(dir_path_wordsim_relatedness_goldstandard , 'r') as file:
    content = file.read()
    lines = content.splitlines()
    for line in lines:
        item_in_line = line.split("\t")
        word_1 = item_in_line[0].lower()
        word_2 = item_in_line[1].lower()
        
        # Cek kata pertama
        try:
            word_vector = model_sg.get_word_vector(word_1)
            if word_1 not in list_words_exist:
                list_words_exist.append(word_1)
        except:
            if word_1 not in list_words_not_exist:
                list_words_not_exist.append(word_1)
        
        # Cek kata kedua
        try:
            word_vector = model_sg.get_word_vector(word_2)
            if word_2 not in list_words_exist:
                list_words_exist.append(word_2)
        except:
            if word_2 not in list_words_not_exist:
                list_words_not_exist.append(word_2)

print("persentase kata ada", len(list_words_exist) / (len(list_words_exist) + len(list_words_not_exist)) )
print("persentase kata tidak ada", len(list_words_not_exist) / (len(list_words_exist) + len(list_words_not_exist)))

print(list_words_exist)
print(list_words_not_exist)

persentase kata ada 1.0
persentase kata tidak ada 0.0
['computer', 'keyboard', 'jerusalem', 'israel', 'planet', 'galaxy', 'canyon', 'landscape', 'opec', 'country', 'day', 'summer', 'dawn', 'citizen', 'people', 'environment', 'ecology', 'maradona', 'football', 'oil', 'money', 'bank', 'software', 'law', 'lawyer', 'weather', 'forecast', 'network', 'hardware', 'nature', 'fbi', 'investigation', 'wealth', 'psychology', 'freud', 'news', 'report', 'war', 'troops', 'physics', 'proton', 'stock', 'market', 'constellation', 'credit', 'card', 'hotel', 'reservation', 'closet', 'clothes', 'soap', 'opera', 'astronomer', 'space', 'movie', 'theater', 'treatment', 'recovery', 'baby', 'mother', 'deposit', 'television', 'film', 'mind', 'game', 'team', 'admission', 'ticket', 'palestinian', 'arafat', 'terror', 'boxing', 'round', 'internet', 'property', 'tennis', 'racket', 'telephone', 'communication', 'currency', 'cognition', 'seafood', 'sea', 'book', 'paper', 'library', 'depression', 'fighting', 'defeating'

In [13]:
# Baca file wordsim_relatedness_goldstandard untuk evaluasi word embedding dengan teknik relatedness cbow
dir_path_wordsim_relatedness_goldstandard = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Relatedness/wordsim_relatedness_goldstandard.txt'
list_model_similarity = []

list_human_score = []
with open(dir_path_wordsim_relatedness_goldstandard , 'r') as file:
    content = file.read()
    lines = content.splitlines()
    for line in lines:
        item_in_line = line.split("\t")
        word_1 = item_in_line[0].lower()
        word_2 = item_in_line[1].lower()
        human_score = float(item_in_line[2])
        try:
            wv_1 = []
            wv_2 = []
            wv_1.append(model_sg.get_word_vector(word_1))
            wv_2.append(model_sg.get_word_vector(word_2))
            similarity = cosine_similarity(wv_1, wv_2)[0][0]
            list_model_similarity.append(similarity)
            list_human_score.append(human_score)
        except:
            continue
            #print("word_1", word_1)
            #print("word_2", word_2)

rho_model, p_model = spearmanr(list_model_similarity, list_human_score)


In [14]:
# Nilai relatedness model sg 1
print(rho_model)

0.1841584927588064


In [13]:
# Nilai word analogy model sg 1
word_analogy(model_sg)

true_predicted 83 0.004246827670896439
false_predicted 19461 0.9957531723291035


In [16]:
# Membuat model word embedding fasttext menggunakan CBOW 2
model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='cbow', lr=0.01, dim=20, ws=5, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0
Progress:  77.5% words/sec/thread:  208668 lr:  0.002245 avg.loss:  0.526140 ETA:   0h10m 6s  0.1% words/sec/thread:  283483 lr:  0.009993 avg.loss:  2.650297 ETA:   0h33m 6s  0.2% words/sec/thread:  278222 lr:  0.009984 avg.loss:  2.462564 ETA:   0h33m42s  0.2% words/sec/thread:  267885 lr:  0.009979 avg.loss:  2.392395 ETA:   0h34m58s  0.7% words/sec/thread:  240422 lr:  0.009932 avg.loss:  2.153061 ETA:   0h38m47s  1.1% words/sec/thread:  232382 lr:  0.009891 avg.loss:  2.059634 ETA:   0h39m58s  1.2% words/sec/thread:  228149 lr:  0.009878 avg.loss:  2.045281 ETA:   0h40m39s  1.3% words/sec/thread:  225079 lr:  0.009869 avg.loss:  2.034234 ETA:   0h41m10s  1.5% words/sec/thread:  224881 lr:  0.009851 avg.loss:  2.018838 ETA:   0h41m 8s  1.8% words/sec/thread:  229334 lr:  0.009823 avg.loss:  1.994506 ETA:   0h40m13s  2.1% words/sec/thread:  232378 lr:  0.009786 avg.loss:  1.967529 ETA:   0h39m32s  2.3% words/sec/thread:  2332

In [17]:
# save model
model_cbow.save_model("model_cbow_fasttext_2.bin")

In [14]:
# load model
model_cbow = fasttext.load_model("model_cbow_fasttext_2.bin")

In [18]:
# Baca file wordsim_relatedness_goldstandard untuk evaluasi word embedding dengan teknik relatedness cbow
dir_path_wordsim_relatedness_goldstandard = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Relatedness/wordsim_relatedness_goldstandard.txt'
list_model_similarity = []

list_human_score = []
with open(dir_path_wordsim_relatedness_goldstandard , 'r') as file:
    content = file.read()
    lines = content.splitlines()
    for line in lines:
        item_in_line = line.split("\t")
        word_1 = item_in_line[0].lower()
        word_2 = item_in_line[1].lower()
        human_score = float(item_in_line[2])
        try:
            wv_1 = []
            wv_2 = []
            wv_1.append(model_cbow.get_word_vector(word_1))
            wv_2.append(model_cbow.get_word_vector(word_2))
            similarity = cosine_similarity(wv_1, wv_2)[0][0]
            list_model_similarity.append(similarity)
            list_human_score.append(human_score)
        except:
            continue
            #print("word_1", word_1)
            #print("word_2", word_2)

rho_model, p_model = spearmanr(list_model_similarity, list_human_score)


In [19]:
# Nilai relatedness model cbow 2
print(rho_model)

0.16123616713210712


In [15]:
# Nilai word analogy model cbow 2
word_analogy(model_cbow)

true_predicted 83 0.004246827670896439
false_predicted 19461 0.9957531723291035


In [20]:
# Membuat model word embedding fasttext menggunakan skip gram 2
model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='skipgram', lr=0.01, dim=20, ws=5, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0


Progress:  32.3% words/sec/thread:   76975 lr:  0.006769 avg.loss:  0.217690 ETA:   1h22m34s  0.1% words/sec/thread:  101108 lr:  0.009993 avg.loss:  2.133708 ETA:   1h32m48s  0.1% words/sec/thread:   98746 lr:  0.009990 avg.loss:  2.113672 ETA:   1h35m 0s  0.2% words/sec/thread:   94325 lr:  0.009983 avg.loss:  2.072477 ETA:   1h39m23s  0.4% words/sec/thread:   82682 lr:  0.009965 avg.loss:  2.061580 ETA:   1h53m10s2.057626 ETA:   1h53m45s  0.5% words/sec/thread:   80889 lr:  0.009951 avg.loss:  2.062291 ETA:   1h55m31s  0.6% words/sec/thread:   78888 lr:  0.009938 avg.loss:  2.069378 ETA:   1h58m18s  0.7% words/sec/thread:   78138 lr:  0.009933 avg.loss:  2.070048 ETA:   1h59m22s  0.7% words/sec/thread:   77827 lr:  0.009928 avg.loss:  2.072261 ETA:   1h59m47s  0.8% words/sec/thread:   77765 lr:  0.009925 avg.loss:  2.073186 ETA:   1h59m51s  0.8% words/sec/thread:   77762 lr:  0.009919 avg.loss:  2.075207 ETA:   1h59m47s  77285 lr:  0.009907 avg.loss:  2.046809 ETA:   2h 0m22s  1.4% 

In [21]:
# save model
model_sg.save_model("model_sg_fasttext_2.bin")

In [16]:
# load model
model_sg = fasttext.load_model("model_sg_fasttext_2.bin")

In [22]:
# Baca file wordsim_relatedness_goldstandard untuk evaluasi word embedding dengan teknik relatedness cbow
dir_path_wordsim_relatedness_goldstandard = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Relatedness/wordsim_relatedness_goldstandard.txt'
list_model_similarity = []

list_human_score = []
with open(dir_path_wordsim_relatedness_goldstandard , 'r') as file:
    content = file.read()
    lines = content.splitlines()
    for line in lines:
        item_in_line = line.split("\t")
        word_1 = item_in_line[0].lower()
        word_2 = item_in_line[1].lower()
        human_score = float(item_in_line[2])
        try:
            wv_1 = []
            wv_2 = []
            wv_1.append(model_sg.get_word_vector(word_1))
            wv_2.append(model_sg.get_word_vector(word_2))
            similarity = cosine_similarity(wv_1, wv_2)[0][0]
            list_model_similarity.append(similarity)
            list_human_score.append(human_score)
        except:
            continue
            #print("word_1", word_1)
            #print("word_2", word_2)

rho_model, p_model = spearmanr(list_model_similarity, list_human_score)


In [23]:
# Nilai relatedness model sg 2
print(rho_model)

0.2732884136555955


In [17]:
# Nilai word analogy model sg 2
word_analogy(model_sg)

true_predicted 47 0.002404830126893164
false_predicted 19497 0.9975951698731068


In [32]:
# Membuat model word embedding fasttext menggunakan CBOW 3
model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='cbow', lr=0.01, dim=30, ws=5, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0


Progress:  35.2% words/sec/thread:  160451 lr:  0.006485 avg.loss:  0.826457 ETA:   0h37m57s 0.009987 avg.loss:  2.504673 ETA:   0h46m12s  0.6% words/sec/thread:  163469 lr:  0.009937 avg.loss:  2.156137 ETA:   0h57m 5s  0.8% words/sec/thread:  156269 lr:  0.009924 avg.loss:  2.114222 ETA:   0h59m38s  0.9% words/sec/thread:  150588 lr:  0.009911 avg.loss:  2.077745 ETA:   1h 1m48s  0.9% words/sec/thread:  148839 lr:  0.009905 avg.loss:  2.062270 ETA:   1h 2m29s  1.0% words/sec/thread:  146785 lr:  0.009898 avg.loss:  2.046132 ETA:   1h 3m19s  1.1% words/sec/thread:  145829 lr:  0.009889 avg.loss:  2.034078 ETA:   1h 3m41s  1.2% words/sec/thread:  145628 lr:  0.009875 avg.loss:  2.015073 ETA:   1h 3m40s  1.3% words/sec/thread:  145838 lr:  0.009868 avg.loss:  2.006980 ETA:   1h 3m32sh 3m12s  1.5% words/sec/thread:  147029 lr:  0.009853 avg.loss:  1.992556 ETA:   1h 2m56s  1.5% words/sec/thread:  146628 lr:  0.009846 avg.loss:  1.985839 ETA:   1h 3m 3s  1.6% words/sec/thread:  145879 lr:

In [33]:
# save model
model_cbow.save_model("model_cbow_fasttext_3.bin")

In [19]:
# load model
model_cbow = fasttext.load_model("model_cbow_fasttext_3.bin")

In [34]:
# Baca file wordsim_relatedness_goldstandard untuk evaluasi word embedding dengan teknik relatedness cbow 3
dir_path_wordsim_relatedness_goldstandard = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Relatedness/wordsim_relatedness_goldstandard.txt'
list_model_similarity = []

list_human_score = []
with open(dir_path_wordsim_relatedness_goldstandard , 'r') as file:
    content = file.read()
    lines = content.splitlines()
    for line in lines:
        item_in_line = line.split("\t")
        word_1 = item_in_line[0].lower()
        word_2 = item_in_line[1].lower()
        human_score = float(item_in_line[2])
        try:
            wv_1 = []
            wv_2 = []
            wv_1.append(model_cbow.get_word_vector(word_1))
            wv_2.append(model_cbow.get_word_vector(word_2))
            similarity = cosine_similarity(wv_1, wv_2)[0][0]
            list_model_similarity.append(similarity)
            list_human_score.append(human_score)
        except:
            continue
            #print("word_1", word_1)
            #print("word_2", word_2)

rho_model, p_model = spearmanr(list_model_similarity, list_human_score)


In [35]:
# Nilai relatedness model cbow 3
print(rho_model)

0.14240675238364528


In [20]:
# Nilai word analogy model cbow 3
word_analogy(model_cbow)

true_predicted 147 0.007521489971346705
false_predicted 19397 0.9924785100286533


In [28]:
# Membuat model word embedding fasttext menggunakan skip gram 3
model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='skipgram', lr=0.01, dim=30, ws=5, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0


Progress:  14.4% words/sec/thread:   38832 lr:  0.008558 avg.loss:  0.459647 ETA:   3h26m57s% words/sec/thread:  101261 lr:  0.009999 avg.loss:  2.905596 ETA:   1h32m43s  0.2% words/sec/thread:   83525 lr:  0.009984 avg.loss:  2.070822 ETA:   1h52m15s  0.2% words/sec/thread:   82432 lr:  0.009980 avg.loss:  2.063923 ETA:   1h53m42s  0.3% words/sec/thread:   76493 lr:  0.009973 avg.loss:  2.060181 ETA:   2h 2m26s  0.3% words/sec/thread:   74674 lr:  0.009970 avg.loss:  2.058096 ETA:   2h 5m23s  0.4% words/sec/thread:   72190 lr:  0.009964 avg.loss:  2.052128 ETA:   2h 9m37s  0.4% words/sec/thread:   68843 lr:  0.009960 avg.loss:  2.048149 ETA:   2h15m51s  0.4% words/sec/thread:   68291 lr:  0.009958 avg.loss:  2.047584 ETA:   2h16m56s  0.5% words/sec/thread:   64661 lr:  0.009949 avg.loss:  2.054654 ETA:   2h24m30s  0.5% words/sec/thread:   64654 lr:  0.009946 avg.loss:  2.055788 ETA:   2h24m27s  0.6% words/sec/thread:   64373 lr:  0.009944 avg.loss:  2.056557 ETA:   2h25m 4s  0.6% word

Progress:  84.9% words/sec/thread:   56497 lr:  0.001506 avg.loss:  0.088966 ETA:   0h25m 1s 80.2% words/sec/thread:   56521 lr:  0.001983 avg.loss:  0.093879 ETA:   0h32m57s 80.2% words/sec/thread:   56522 lr:  0.001979 avg.loss:  0.093830 ETA:   0h32m52s 80.3% words/sec/thread:   56523 lr:  0.001970 avg.loss:  0.093731 ETA:   0h32m43s 80.3% words/sec/thread:   56521 lr:  0.001966 avg.loss:  0.093685 ETA:   0h32m39s 80.4% words/sec/thread:   56518 lr:  0.001962 avg.loss:  0.093645 ETA:   0h32m36s 80.4% words/sec/thread:   56519 lr:  0.001958 avg.loss:  0.093606 ETA:   0h32m32s 80.5% words/sec/thread:   56520 lr:  0.001953 avg.loss:  0.093545 ETA:   0h32m26s 80.5% words/sec/thread:   56515 lr:  0.001945 avg.loss:  0.093466 ETA:   0h32m19s 0.001940 avg.loss:  0.093401 ETA:   0h32m13s 80.6% words/sec/thread:   56516 lr:  0.001937 avg.loss:  0.093369 ETA:   0h32m10s 80.7% words/sec/thread:   56516 lr:  0.001935 avg.loss:  0.093346 ETA:   0h32m 8s 80.8% words/sec/thread:   56516 lr:  0.001

Progress:  92.3% words/sec/thread:   56816 lr:  0.000770 avg.loss:  0.082434 ETA:   0h12m43s 85.0% words/sec/thread:   56497 lr:  0.001504 avg.loss:  0.088954 ETA:   0h25m 0s 85.0% words/sec/thread:   56498 lr:  0.001502 avg.loss:  0.088927 ETA:   0h24m57s 85.1% words/sec/thread:   56494 lr:  0.001487 avg.loss:  0.088789 ETA:   0h24m43s85.1% words/sec/thread:   56495 lr:  0.001486 avg.loss:  0.088773 ETA:   0h24m41s 85.2% words/sec/thread:   56497 lr:  0.001479 avg.loss:  0.088714 ETA:   0h24m35s 85.2% words/sec/thread:   56496 lr:  0.001475 avg.loss:  0.088673 ETA:   0h24m31s 85.3% words/sec/thread:   56496 lr:  0.001474 avg.loss:  0.088658 ETA:   0h24m29s 85.4% words/sec/thread:   56501 lr:  0.001464 avg.loss:  0.088564 ETA:   0h24m19s 85.4% words/sec/thread:   56500 lr:  0.001461 avg.loss:  0.088538 ETA:   0h24m17s 85.4% words/sec/thread:   56500 lr:  0.001457 avg.loss:  0.088498 ETA:   0h24m13s 85.5% words/sec/thread:   56500 lr:  0.001450 avg.loss:  0.088427 ETA:   0h24m 5s 85.5% 

Progress:  99.1% words/sec/thread:   57175 lr:  0.000089 avg.loss:  0.077291 ETA:   0h 1m27s 92.5% words/sec/thread:   56835 lr:  0.000747 avg.loss:  0.082248 ETA:   0h12m20s 92.6% words/sec/thread:   56838 lr:  0.000740 avg.loss:  0.082190 ETA:   0h12m13s 92.6% words/sec/thread:   56841 lr:  0.000738 avg.loss:  0.082169 ETA:   0h12m11s 92.7% words/sec/thread:   56847 lr:  0.000732 avg.loss:  0.082124 ETA:   0h12m 5s 92.7% words/sec/thread:   56852 lr:  0.000726 avg.loss:  0.082075 ETA:   0h11m59s 92.8% words/sec/thread:   56856 lr:  0.000723 avg.loss:  0.082047 ETA:   0h11m56s 92.9% words/sec/thread:   56870 lr:  0.000711 avg.loss:  0.081958 ETA:   0h11m44s 92.9% words/sec/thread:   56875 lr:  0.000706 avg.loss:  0.081913 ETA:   0h11m39s 93.0% words/sec/thread:   56881 lr:  0.000701 avg.loss:  0.081873 ETA:   0h11m33s 93.0% words/sec/thread:   56885 lr:  0.000697 avg.loss:  0.081844 ETA:   0h11m30s 93.1% words/sec/thread:   56891 lr:  0.000688 avg.loss:  0.081777 ETA:   0h11m21s lr:  

Progress: 100.0% words/sec/thread:   57216 lr:  0.000000 avg.loss:  0.076661 ETA:   0h 0m 0s 99.1% words/sec/thread:   57175 lr:  0.000088 avg.loss:  0.077283 ETA:   0h 1m26s 99.1% words/sec/thread:   57175 lr:  0.000086 avg.loss:  0.077269 ETA:   0h 1m24s lr:  0.000084 avg.loss:  0.077254 ETA:   0h 1m22s 99.2% words/sec/thread:   57176 lr:  0.000078 avg.loss:  0.077213 ETA:   0h 1m17s 99.3% words/sec/thread:   57178 lr:  0.000072 avg.loss:  0.077167 ETA:   0h 1m10s 99.3% words/sec/thread:   57179 lr:  0.000070 avg.loss:  0.077159 ETA:   0h 1m 9s 99.4% words/sec/thread:   57187 lr:  0.000057 avg.loss:  0.077066 ETA:   0h 0m56s 99.5% words/sec/thread:   57195 lr:  0.000049 avg.loss:  0.077008 ETA:   0h 0m48s 99.6% words/sec/thread:   57199 lr:  0.000043 avg.loss:  0.076966 ETA:   0h 0m42s


In [29]:
# save model
model_sg.save_model("model_sg_fasttext_3.bin")

In [21]:
# load model
model_sg = fasttext.load_model("model_sg_fasttext_3.bin")

In [30]:
# Baca file wordsim_relatedness_goldstandard untuk evaluasi word embedding dengan teknik relatedness sg 3
dir_path_wordsim_relatedness_goldstandard = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Relatedness/wordsim_relatedness_goldstandard.txt'
list_model_similarity = []

list_human_score = []
with open(dir_path_wordsim_relatedness_goldstandard , 'r') as file:
    content = file.read()
    lines = content.splitlines()
    for line in lines:
        item_in_line = line.split("\t")
        word_1 = item_in_line[0].lower()
        word_2 = item_in_line[1].lower()
        human_score = float(item_in_line[2])
        try:
            wv_1 = []
            wv_2 = []
            wv_1.append(model_sg.get_word_vector(word_1))
            wv_2.append(model_sg.get_word_vector(word_2))
            similarity = cosine_similarity(wv_1, wv_2)[0][0]
            list_model_similarity.append(similarity)
            list_human_score.append(human_score)
        except:
            continue
            #print("word_1", word_1)
            #print("word_2", word_2)

rho_model, p_model = spearmanr(list_model_similarity, list_human_score)


In [31]:
# Nilai relatedness model sg 3
print(rho_model)

0.26821201933712185


In [22]:
# Nilai word analogy model sg 3
word_analogy(model_sg)

true_predicted 79 0.004042161277118297
false_predicted 19465 0.9959578387228817


In [8]:
# Membuat model word embedding fasttext menggunakan CBOW 4
model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='cbow', lr=0.01, dim=20, ws=5, epoch=5000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0


Progress:   3.7% words/sec/thread:  204622 lr:  0.009634 avg.loss:  1.242080 ETA:   3h41m 4s  0.0% words/sec/thread:  261443 lr:  0.009999 avg.loss:  2.710490 ETA:   2h59m35s  0.0% words/sec/thread:  259792 lr:  0.009997 avg.loss:  2.475919 ETA:   3h 0m41s  0.1% words/sec/thread:  247171 lr:  0.009994 avg.loss:  2.319847 ETA:   3h 9m51s  0.1% words/sec/thread:  241989 lr:  0.009992 avg.loss:  2.261672 ETA:   3h13m53s  0.1% words/sec/thread:  235918 lr:  0.009988 avg.loss:  2.176712 ETA:   3h18m48s  0.1% words/sec/thread:  228701 lr:  0.009987 avg.loss:  2.157590 ETA:   3h25m 3s  0.2% words/sec/thread:  215874 lr:  0.009983 avg.loss:  2.101743 ETA:   3h37m 9s  0.2% words/sec/thread:  213881 lr:  0.009980 avg.loss:  2.067180 ETA:   3h39m 6s  0.2% words/sec/thread:  212505 lr:  0.009977 avg.loss:  2.045678 ETA:   3h40m27s  0.3% words/sec/thread:  211002 lr:  0.009973 avg.loss:  2.027250 ETA:   3h41m56s  0.3% words/sec/thread:  207996 lr:  0.009970 avg.loss:  2.011879 ETA:   3h45m 4s  0.3%

Progress:  15.9% words/sec/thread:  184760 lr:  0.008410 avg.loss:  0.535641 ETA:   3h33m44s13.4% words/sec/thread:  188912 lr:  0.008660 avg.loss:  0.581851 ETA:   3h35m14s 13.4% words/sec/thread:  188878 lr:  0.008659 avg.loss:  0.581606 ETA:   3h35m15s 13.5% words/sec/thread:  188755 lr:  0.008655 avg.loss:  0.580768 ETA:   3h35m18s 188674 lr:  0.008653 avg.loss:  0.580479 ETA:   3h35m21s 13.5% words/sec/thread:  188642 lr:  0.008653 avg.loss:  0.580346 ETA:   3h35m22s 13.5% words/sec/thread:  188610 lr:  0.008652 avg.loss:  0.580205 ETA:   3h35m23s 13.5% words/sec/thread:  188544 lr:  0.008649 avg.loss:  0.579788 ETA:   3h35m24s 13.5% words/sec/thread:  188491 lr:  0.008648 avg.loss:  0.579493 ETA:   3h35m25s 188456 lr:  0.008646 avg.loss:  0.579157 ETA:   3h35m25s 13.6% words/sec/thread:  188431 lr:  0.008644 avg.loss:  0.578867 ETA:   3h35m24s% words/sec/thread:  188416 lr:  0.008643 avg.loss:  0.578697 ETA:   3h35m24s 13.6% words/sec/thread:  188310 lr:  0.008640 avg.loss:  0.57

Progress:  19.1% words/sec/thread:  181694 lr:  0.008092 avg.loss:  0.451168 ETA:   3h29m 7s 15.9% words/sec/thread:  184759 lr:  0.008410 avg.loss:  0.535410 ETA:   3h33m43s 15.9% words/sec/thread:  184754 lr:  0.008409 avg.loss:  0.535095 ETA:   3h33m42s 15.9% words/sec/thread:  184748 lr:  0.008408 avg.loss:  0.534784 ETA:   3h33m41s 15.9% words/sec/thread:  184746 lr:  0.008406 avg.loss:  0.534224 ETA:   3h33m38s 16.0% words/sec/thread:  184744 lr:  0.008405 avg.loss:  0.533889 ETA:   3h33m37s 0.531475 ETA:   3h33m31s 16.1% words/sec/thread:  184637 lr:  0.008394 avg.loss:  0.530623 ETA:   3h33m28s 16.1% words/sec/thread:  184625 lr:  0.008391 avg.loss:  0.529748 ETA:   3h33m25s 16.1% words/sec/thread:  184608 lr:  0.008389 avg.loss:  0.528929 ETA:   3h33m22s 16.1% words/sec/thread:  184547 lr:  0.008385 avg.loss:  0.527908 ETA:   3h33m21s avg.loss:  0.527628 ETA:   3h33m20s 16.2% words/sec/thread:  184525 lr:  0.008381 avg.loss:  0.526555 ETA:   3h33m16s 16.2% words/sec/thread:  1

Progress:  22.0% words/sec/thread:  170839 lr:  0.007796 avg.loss:  0.394897 ETA:   3h34m16s 19.1% words/sec/thread:  181672 lr:  0.008089 avg.loss:  0.450610 ETA:   3h29m 5s 19.1% words/sec/thread:  181654 lr:  0.008088 avg.loss:  0.450320 ETA:   3h29m 4s 19.1% words/sec/thread:  181637 lr:  0.008086 avg.loss:  0.449929 ETA:   3h29m 2s 19.2% words/sec/thread:  181604 lr:  0.008084 avg.loss:  0.449374 ETA:   3h29m 0s 19.3% words/sec/thread:  181229 lr:  0.008075 avg.loss:  0.447376 ETA:   3h29m12s words/sec/thread:  180914 lr:  0.008052 avg.loss:  0.442515 ETA:   3h28m59s 19.5% words/sec/thread:  180934 lr:  0.008047 avg.loss:  0.441610 ETA:   3h28m50s 19.6% words/sec/thread:  180936 lr:  0.008044 avg.loss:  0.440910 ETA:   3h28m45s ETA:   3h28m44s 19.6% words/sec/thread:  180901 lr:  0.008040 avg.loss:  0.440071 ETA:   3h28m41s 19.6% words/sec/thread:  180896 lr:  0.008038 avg.loss:  0.439709 ETA:   3h28m39s 19.6% words/sec/thread:  180884 lr:  0.008036 avg.loss:  0.439105 ETA:   3h28

...

Progress:  79.5% words/sec/thread:  178846 lr:  0.002052 avg.loss:  0.133479 ETA:   0h53m52s 73.6% words/sec/thread:  179240 lr:  0.002639 avg.loss:  0.142037 ETA:   1h 9m 7s avg.loss:  0.142009 ETA:   1h 9m 5s 73.6% words/sec/thread:  179248 lr:  0.002636 avg.loss:  0.141993 ETA:   1h 9m 3s 73.6% words/sec/thread:  179249 lr:  0.002636 avg.loss:  0.141990 ETA:   1h 9m 3s 73.7% words/sec/thread:  179252 lr:  0.002634 avg.loss:  0.141961 ETA:   1h 9m 0s 73.7% words/sec/thread:  179247 lr:  0.002629 avg.loss:  0.141879 ETA:   1h 8m52s 73.8% words/sec/thread:  179252 lr:  0.002623 avg.loss:  0.141780 ETA:   1h 8m41s 73.8% words/sec/thread:  179267 lr:  0.002620 avg.loss:  0.141734 ETA:   1h 8m36s 73.9% words/sec/thread:  179279 lr:  0.002613 avg.loss:  0.141628 ETA:   1h 8m26s 73.9% words/sec/thread:  179290 lr:  0.002608 avg.loss:  0.141555 ETA:   1h 8m18s 74.0% words/sec/thread:  179302 lr:  0.002604 avg.loss:  0.141489 ETA:   1h 8m11s 74.0% words/sec/thread:  179312 lr:  0.002602 avg.l

Progress:  84.8% words/sec/thread:  179054 lr:  0.001525 avg.loss:  0.126665 ETA:   0h39m59s 79.5% words/sec/thread:  178841 lr:  0.002047 avg.loss:  0.133424 ETA:   0h53m45s 79.6% words/sec/thread:  178839 lr:  0.002041 avg.loss:  0.133334 ETA:   0h53m35s 79.6% words/sec/thread:  178837 lr:  0.002036 avg.loss:  0.133266 ETA:   0h53m26s 79.7% words/sec/thread:  178839 lr:  0.002032 avg.loss:  0.133224 ETA:   0h53m21s 79.7% words/sec/thread:  178842 lr:  0.002029 avg.loss:  0.133180 ETA:   0h53m16s 79.7% words/sec/thread:  178842 lr:  0.002027 avg.loss:  0.133158 ETA:   0h53m13s words/sec/thread:  178841 lr:  0.002024 avg.loss:  0.133118 ETA:   0h53m 9s 79.8% words/sec/thread:  178842 lr:  0.002021 avg.loss:  0.133072 ETA:   0h53m 3s 79.8% words/sec/thread:  178843 lr:  0.002020 avg.loss:  0.133053 ETA:   0h53m 1s 79.9% words/sec/thread:  178837 lr:  0.002014 avg.loss:  0.132983 ETA:   0h52m53s 79.9% words/sec/thread:  178831 lr:  0.002012 avg.loss:  0.132945 ETA:   0h52m49s 80.0% words

Progress:  92.0% words/sec/thread:  178622 lr:  0.000797 avg.loss:  0.118218 ETA:   0h20m57s 84.8% words/sec/thread:  179062 lr:  0.001523 avg.loss:  0.126637 ETA:   0h39m55s 84.8% words/sec/thread:  179067 lr:  0.001520 avg.loss:  0.126604 ETA:   0h39m51s 84.8% words/sec/thread:  179081 lr:  0.001516 avg.loss:  0.126552 ETA:   0h39m44s 84.9% words/sec/thread:  179091 lr:  0.001512 avg.loss:  0.126502 ETA:   0h39m37s 0.001507 avg.loss:  0.126447 ETA:   0h39m31s 85.0% words/sec/thread:  179088 lr:  0.001505 avg.loss:  0.126417 ETA:   0h39m27s 85.0% words/sec/thread:  179080 lr:  0.001495 avg.loss:  0.126303 ETA:   0h39m12s85.1% words/sec/thread:  179082 lr:  0.001494 avg.loss:  0.126286 ETA:   0h39m10s85.1% words/sec/thread:  179088 lr:  0.001490 avg.loss:  0.126231 ETA:   0h39m 3s 85.2% words/sec/thread:  179091 lr:  0.001480 avg.loss:  0.126108 ETA:   0h38m47s 85.2% words/sec/thread:  179096 lr:  0.001477 avg.loss:  0.126071 ETA:   0h38m42s 85.3% words/sec/thread:  179108 lr:  0.00147

Progress:  99.0% words/sec/thread:  179950 lr:  0.000099 avg.loss:  0.110867 ETA:   0h 2m34sh20m55s 92.0% words/sec/thread:  178626 lr:  0.000796 avg.loss:  0.118200 ETA:   0h20m54s 0.000792 avg.loss:  0.118154 ETA:   0h20m48s 92.1% words/sec/thread:  178623 lr:  0.000787 avg.loss:  0.118103 ETA:   0h20m40s 92.2% words/sec/thread:  178624 lr:  0.000783 avg.loss:  0.118058 ETA:   0h20m34s 92.2% words/sec/thread:  178625 lr:  0.000778 avg.loss:  0.118008 ETA:   0h20m27s 92.2% words/sec/thread:  178631 lr:  0.000775 avg.loss:  0.117978 ETA:   0h20m22s 92.3% words/sec/thread:  178640 lr:  0.000767 avg.loss:  0.117891 ETA:   0h20m10s avg.loss:  0.117870 ETA:   0h20m 7s 92.4% words/sec/thread:  178650 lr:  0.000763 avg.loss:  0.117843 ETA:   0h20m 3s% words/sec/thread:  178662 lr:  0.000759 avg.loss:  0.117803 ETA:   0h19m57s 92.4% words/sec/thread:  178665 lr:  0.000756 avg.loss:  0.117769 ETA:   0h19m52s 92.5% words/sec/thread:  178671 lr:  0.000754 avg.loss:  0.117741 ETA:   0h19m48s 92.5

Progress: 100.0% words/sec/thread:  180233 lr:  0.000000 avg.loss:  0.109871 ETA:   0h 0m 0s 99.0% words/sec/thread:  179954 lr:  0.000098 avg.loss:  0.110852 ETA:   0h 2m32s 99.1% words/sec/thread:  179971 lr:  0.000093 avg.loss:  0.110811 ETA:   0h 2m26s 99.1% words/sec/thread:  179980 lr:  0.000090 avg.loss:  0.110781 ETA:   0h 2m21s 99.1% words/sec/thread:  179985 lr:  0.000089 avg.loss:  0.110770 ETA:   0h 2m19s 99.2% words/sec/thread:  180004 lr:  0.000081 avg.loss:  0.110689 ETA:   0h 2m 7s 99.2% words/sec/thread:  180015 lr:  0.000076 avg.loss:  0.110631 ETA:   0h 1m58s 99.3% words/sec/thread:  180025 lr:  0.000072 avg.loss:  0.110599 ETA:   0h 1m53s 99.3% words/sec/thread:  180033 lr:  0.000068 avg.loss:  0.110553 ETA:   0h 1m46s99.3% words/sec/thread:  180038 lr:  0.000066 avg.loss:  0.110534 ETA:   0h 1m43s 99.5% words/sec/thread:  180072 lr:  0.000054 avg.loss:  0.110410 ETA:   0h 1m24s 99.5% words/sec/thread:  180077 lr:  0.000052 avg.loss:  0.110395 ETA:   0h 1m21s 180085

In [9]:
# save model
model_cbow.save_model("model_cbow_fasttext_4.bin")

In [23]:
# load model
model_cbow = fasttext.load_model("model_cbow_fasttext_4.bin")

In [10]:
# Baca file wordsim_relatedness_goldstandard untuk evaluasi word embedding dengan teknik relatedness cbow 4
dir_path_wordsim_relatedness_goldstandard = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Relatedness/wordsim_relatedness_goldstandard.txt'
list_model_similarity = []

list_human_score = []
with open(dir_path_wordsim_relatedness_goldstandard , 'r') as file:
    content = file.read()
    lines = content.splitlines()
    for line in lines:
        item_in_line = line.split("\t")
        word_1 = item_in_line[0].lower()
        word_2 = item_in_line[1].lower()
        human_score = float(item_in_line[2])
        try:
            wv_1 = []
            wv_2 = []
            wv_1.append(model_cbow.get_word_vector(word_1))
            wv_2.append(model_cbow.get_word_vector(word_2))
            similarity = cosine_similarity(wv_1, wv_2)[0][0]
            list_model_similarity.append(similarity)
            list_human_score.append(human_score)
        except:
            continue
            #print("word_1", word_1)
            #print("word_2", word_2)

rho_model, p_model = spearmanr(list_model_similarity, list_human_score)


In [11]:
# Nilai relatedness model cbow 4
print(rho_model)

0.2181503500691083


In [24]:
# Nilai word analogy model cbow 4
word_analogy(model_cbow)

true_predicted 74 0.00378632828489562
false_predicted 19470 0.9962136717151043


In [12]:
# Membuat model word embedding fasttext menggunakan skip gram 4
model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='skipgram', lr=0.01, dim=20, ws=5, epoch=5000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0


Progress:   1.6% words/sec/thread:   66410 lr:  0.009835 avg.loss:  0.658034 ETA:  11h35m24s  0.0% words/sec/thread:  106142 lr:  0.010000 avg.loss:  2.388967 ETA:   7h22m22sh 2m18s 0.009993 avg.loss:  2.054973 ETA:   9h 8m28s  0.1% words/sec/thread:   81482 lr:  0.009992 avg.loss:  2.051919 ETA:   9h35m48sh41m 0s  0.1% words/sec/thread:   79411 lr:  0.009989 avg.loss:  2.060874 ETA:   9h50m39s  0.1% words/sec/thread:   75996 lr:  0.009988 avg.loss:  2.065898 ETA:  10h17m 7s  0.1% words/sec/thread:   72160 lr:  0.009987 avg.loss:  2.067856 ETA:  10h49m52s0.009987 avg.loss:  2.069000 ETA:  10h57m42s  0.1% words/sec/thread:   69554 lr:  0.009986 avg.loss:  2.070590 ETA:  11h14m10s  0.2% words/sec/thread:   67087 lr:  0.009985 avg.loss:  2.071586 ETA:  11h38m51s  0.2% words/sec/thread:   66250 lr:  0.009984 avg.loss:  2.073674 ETA:  11h47m38s  0.2% words/sec/thread:   66639 lr:  0.009983 avg.loss:  2.075097 ETA:  11h43m25savg.loss:  2.071454 ETA:  11h41m37s% words/sec/thread:   66168 lr: 

Progress:  24.3% words/sec/thread:   52630 lr:  0.007574 avg.loss:  0.066000 ETA:  11h15m44s 18.3% words/sec/thread:   55653 lr:  0.008165 avg.loss:  0.084527 ETA:  11h28m54s 18.4% words/sec/thread:   55593 lr:  0.008157 avg.loss:  0.084196 ETA:  11h28m58s 0.083977 ETA:  11h28m50s 18.5% words/sec/thread:   55542 lr:  0.008147 avg.loss:  0.083784 ETA:  11h28m45s 18.7% words/sec/thread:   55395 lr:  0.008126 avg.loss:  0.082947 ETA:  11h28m47s 18.8% words/sec/thread:   55381 lr:  0.008124 avg.loss:  0.082864 ETA:  11h28m47s 18.8% words/sec/thread:   55370 lr:  0.008123 avg.loss:  0.082827 ETA:  11h28m51s% words/sec/thread:   55339 lr:  0.008117 avg.loss:  0.082608 ETA:  11h28m46s% words/sec/thread:   55272 lr:  0.008105 avg.loss:  0.082115 ETA:  11h28m31s 19.0% words/sec/thread:   55245 lr:  0.008101 avg.loss:  0.081987 ETA:  11h28m34s 19.0% words/sec/thread:   55229 lr:  0.008099 avg.loss:  0.081892 ETA:  11h28m33s 19.0% words/sec/thread:   55220 lr:  0.008098 avg.loss:  0.081857 ETA:  

Progress:  29.4% words/sec/thread:   50938 lr:  0.007060 avg.loss:  0.056071 ETA:  10h50m50s24.3% words/sec/thread:   52622 lr:  0.007572 avg.loss:  0.065947 ETA:  11h15m39s 24.3% words/sec/thread:   52616 lr:  0.007570 avg.loss:  0.065899 ETA:  11h15m32sh15m 2s 24.5% words/sec/thread:   52540 lr:  0.007549 avg.loss:  0.065394 ETA:  11h14m37s 24.6% words/sec/thread:   52516 lr:  0.007540 avg.loss:  0.065198 ETA:  11h14m11s 24.7% words/sec/thread:   52453 lr:  0.007526 avg.loss:  0.064858 ETA:  11h13m41s 24.8% words/sec/thread:   52451 lr:  0.007525 avg.loss:  0.064845 ETA:  11h13m39s words/sec/thread:   52440 lr:  0.007522 avg.loss:  0.064781 ETA:  11h13m33s% words/sec/thread:   52437 lr:  0.007521 avg.loss:  0.064763 ETA:  11h13m31s 24.8% words/sec/thread:   52432 lr:  0.007520 avg.loss:  0.064718 ETA:  11h13m24s 24.9% words/sec/thread:   52397 lr:  0.007507 avg.loss:  0.064422 ETA:  11h12m43s 25.1% words/sec/thread:   52335 lr:  0.007491 avg.loss:  0.064073 ETA:  11h12m 9s9s 25.2% wo

Progress:  35.4% words/sec/thread:   49659 lr:  0.006461 avg.loss:  0.048062 ETA:  10h10m54s 29.5% words/sec/thread:   50912 lr:  0.007049 avg.loss:  0.055878 ETA:  10h50m 5s% words/sec/thread:   50903 lr:  0.007044 avg.loss:  0.055805 ETA:  10h49m46s 0.055788 ETA:  10h49m43s29.6% words/sec/thread:   50898 lr:  0.007043 avg.loss:  0.055786 ETA:  10h49m43s 29.6% words/sec/thread:   50883 lr:  0.007036 avg.loss:  0.055675 ETA:  10h49m16s 29.7% words/sec/thread:   50881 lr:  0.007034 avg.loss:  0.055643 ETA:  10h49m 7s 29.7% words/sec/thread:   50855 lr:  0.007026 avg.loss:  0.055519 ETA:  10h48m42s 0.055429 ETA:  10h48m24s 29.8% words/sec/thread:   50831 lr:  0.007018 avg.loss:  0.055406 ETA:  10h48m19s avg.loss:  0.055254 ETA:  10h47m41s 29.9% words/sec/thread:   50808 lr:  0.007008 avg.loss:  0.055247 ETA:  10h47m39s 30.1% words/sec/thread:   50766 lr:  0.006992 avg.loss:  0.055008 ETA:  10h46m43s0.006991 avg.loss:  0.055000 ETA:  10h46m42s30.1% words/sec/thread:   50761 lr:  0.006991 

...

Progress:  91.5% words/sec/thread:   45521 lr:  0.000847 avg.loss:  0.023889 ETA:   1h27m25s 89.8% words/sec/thread:   46600 lr:  0.001019 avg.loss:  0.024191 ETA:   1h42m42s 0.024188 ETA:   1h42m31sh42m31s 89.8% words/sec/thread:   46592 lr:  0.001017 avg.loss:  0.024188 ETA:   1h42m31s 89.8% words/sec/thread:   46592 lr:  0.001017 avg.loss:  0.024188 ETA:   1h42m30s 89.8% words/sec/thread:   46585 lr:  0.001015 avg.loss:  0.024184 ETA:   1h42m20s 89.9% words/sec/thread:   46571 lr:  0.001011 avg.loss:  0.024176 ETA:   1h41m58ss  46550 lr:  0.001007 avg.loss:  0.024169 ETA:   1h41m36s 0.024169 ETA:   1h41m36s ETA:   1h41m29s ETA:   1h41m27s ETA:   1h41m22s 90.0% words/sec/thread:   46525 lr:  0.001004 avg.loss:  0.024163 ETA:   1h41m16s 90.0% words/sec/thread:   46518 lr:  0.001002 avg.loss:  0.024160 ETA:   1h41m 8s% words/sec/thread:   46516 lr:  0.001002 avg.loss:  0.024160 ETA:   1h41m 6s 90.0% words/sec/thread:   46509 lr:  0.001000 avg.loss:  0.024157 ETA:   1h41m 0s90.0% words/

Progress:  95.2% words/sec/thread:   43638 lr:  0.000479 avg.loss:  0.023273 ETA:   0h51m32s 91.6% words/sec/thread:   45498 lr:  0.000843 avg.loss:  0.023882 ETA:   1h26m59s 91.6% words/sec/thread:   45497 lr:  0.000843 avg.loss:  0.023882 ETA:   1h26m58s 91.6% words/sec/thread:   45475 lr:  0.000839 avg.loss:  0.023875 ETA:   1h26m37s  45469 lr:  0.000838 avg.loss:  0.023874 ETA:   1h26m34s 91.6% words/sec/thread:   45466 lr:  0.000838 avg.loss:  0.023874 ETA:   1h26m32s 91.6% words/sec/thread:   45465 lr:  0.000838 avg.loss:  0.023873 ETA:   1h26m31s 91.6% words/sec/thread:   45464 lr:  0.000838 avg.loss:  0.023873 ETA:   1h26m31s 91.6% words/sec/thread:   45463 lr:  0.000838 avg.loss:  0.023873 ETA:   1h26m30s 91.7% words/sec/thread:   45446 lr:  0.000834 avg.loss:  0.023866 ETA:   1h26m 8s4s 91.7% words/sec/thread:   45440 lr:  0.000833 avg.loss:  0.023864 ETA:   1h26m 4s26m 1s91.7% words/sec/thread:   45436 lr:  0.000832 avg.loss:  0.023863 ETA:   1h26m 1s45409 lr:  0.000824 avg.

Progress:  97.4% words/sec/thread:   42868 lr:  0.000261 avg.loss:  0.022930 ETA:   0h28m34s  43622 lr:  0.000473 avg.loss:  0.023263 ETA:   0h50m53s95.3% words/sec/thread:   43612 lr:  0.000469 avg.loss:  0.023256 ETA:   0h50m31s95.3% words/sec/thread:   43607 lr:  0.000467 avg.loss:  0.023254 ETA:   0h50m20s 95.3% words/sec/thread:   43605 lr:  0.000467 avg.loss:  0.023252 ETA:   0h50m15s 95.3% words/sec/thread:   43605 lr:  0.000467 avg.loss:  0.023252 ETA:   0h50m15s50m13s 95.3% words/sec/thread:   43601 lr:  0.000466 avg.loss:  0.023251 ETA:   0h50m 9s 9s 95.4% words/sec/thread:   43592 lr:  0.000462 avg.loss:  0.023246 ETA:   0h49m47s 95.4% words/sec/thread:   43589 lr:  0.000461 avg.loss:  0.023244 ETA:   0h49m41s 95.5% words/sec/thread:   43567 lr:  0.000454 avg.loss:  0.023233 ETA:   0h48m55s 95.6% words/sec/thread:   43540 lr:  0.000444 avg.loss:  0.023216 ETA:   0h47m51s95.6% words/sec/thread:   43530 lr:  0.000440 avg.loss:  0.023210 ETA:   0h47m27s47m23s 95.7% words/sec/th

Progress:  99.6% words/sec/thread:   41792 lr:  0.000045 avg.loss:  0.022604 ETA:   0h 5m 2s% words/sec/thread:   42865 lr:  0.000260 avg.loss:  0.022929 ETA:   0h28m30s% words/sec/thread:   42854 lr:  0.000258 avg.loss:  0.022925 ETA:   0h28m16s 97.4% words/sec/thread:   42853 lr:  0.000258 avg.loss:  0.022925 ETA:   0h28m15s 97.4% words/sec/thread:   42852 lr:  0.000258 avg.loss:  0.022925 ETA:   0h28m14s 97.4% words/sec/thread:   42850 lr:  0.000257 avg.loss:  0.022924 ETA:   0h28m11s 0.000256 avg.loss:  0.022923 ETA:   0h28m 5s 97.4% words/sec/thread:   42840 lr:  0.000255 avg.loss:  0.022922 ETA:   0h27m58ss  0h27m41s 97.5% words/sec/thread:   42802 lr:  0.000248 avg.loss:  0.022910 ETA:   0h27m14s 97.5% words/sec/thread:   42797 lr:  0.000247 avg.loss:  0.022908 ETA:   0h27m 5s0s 97.6% words/sec/thread:   42779 lr:  0.000243 avg.loss:  0.022902 ETA:   0h26m41s 97.6% words/sec/thread:   42776 lr:  0.000243 avg.loss:  0.022901 ETA:   0h26m37ss0.000241 avg.loss:  0.022899 ETA:   0h2

Progress: 100.0% words/sec/thread:   41537 lr:  0.000000 avg.loss:  0.022538 ETA:   0h 0m 0s 99.6% words/sec/thread:   41785 lr:  0.000043 avg.loss:  0.022601 ETA:   0h 4m52s 99.6% words/sec/thread:   41776 lr:  0.000042 avg.loss:  0.022599 ETA:   0h 4m40s41757 lr:  0.000039 avg.loss:  0.022595 ETA:   0h 4m21s 99.6% words/sec/thread:   41756 lr:  0.000039 avg.loss:  0.022595 ETA:   0h 4m20s 99.6% words/sec/thread:   41751 lr:  0.000038 avg.loss:  0.022594 ETA:   0h 4m15s% words/sec/thread:   41749 lr:  0.000038 avg.loss:  0.022594 ETA:   0h 4m14ss99.7% words/sec/thread:   41727 lr:  0.000034 avg.loss:  0.022589 ETA:   0h 3m51s 99.7% words/sec/thread:   41725 lr:  0.000034 avg.loss:  0.022589 ETA:   0h 3m49s words/sec/thread:   41716 lr:  0.000033 avg.loss:  0.022586 ETA:   0h 3m40s 99.7% words/sec/thread:   41700 lr:  0.000030 avg.loss:  0.022583 ETA:   0h 3m24s 99.7% words/sec/thread:   41696 lr:  0.000030 avg.loss:  0.022581 ETA:   0h 3m20s 99.7% words/sec/thread:   41691 lr:  0.0000

In [13]:
# save model
model_sg.save_model("model_sg_fasttext_4.bin")

In [25]:
# load model
model_sg = fasttext.load_model("model_sg_fasttext_4.bin")

In [5]:
# Baca file wordsim_relatedness_goldstandard untuk evaluasi word embedding dengan teknik relatedness sg 4
dir_path_wordsim_relatedness_goldstandard = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Relatedness/wordsim_relatedness_goldstandard.txt'
list_model_similarity = []

list_human_score = []
with open(dir_path_wordsim_relatedness_goldstandard , 'r') as file:
    content = file.read()
    lines = content.splitlines()
    for line in lines:
        item_in_line = line.split("\t")
        word_1 = item_in_line[0].lower()
        word_2 = item_in_line[1].lower()
        human_score = float(item_in_line[2])
        try:
            wv_1 = []
            wv_2 = []
            wv_1.append(model_sg.get_word_vector(word_1))
            wv_2.append(model_sg.get_word_vector(word_2))
            similarity = cosine_similarity(wv_1, wv_2)[0][0]
            list_model_similarity.append(similarity)
            list_human_score.append(human_score)
        except:
            continue
            #print("word_1", word_1)
            #print("word_2", word_2)

rho_model, p_model = spearmanr(list_model_similarity, list_human_score)


In [6]:
# Nilai relatedness model sg 4
print(rho_model)

0.27716235605152345


In [26]:
# Nilai word analogy model sg 4
word_analogy(model_sg)

true_predicted 59 0.003018829308227589
false_predicted 19485 0.9969811706917724


In [ ]:
# Baca file word_analogy untuk evaluasi word embedding dengan teknik word analogy (menggunakan cara manual) (tidak dipake karena terlalu lama)

dir_path_word_analogy = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Word_analogy/Word_Analogy.txt'

true_predicted = 0
false_predicted = 0

list_of_words = model_sg.words


with open(dir_path_word_analogy , 'r') as file:
    content = file.read()
    lines = content.splitlines() 
    for line in lines:
        item_in_line = line.split(" ")
        if len(item_in_line) == 4: # untuk buang baris yang merupakan keterangan
            max_value = -sys.maxsize - 1 # diisi dengan min value dahulu
            word_1 = item_in_line[0].lower()
            word_2 = item_in_line[1].lower()
            word_3 = item_in_line[2].lower()
            word_4 = item_in_line[3].lower()
            try:
                wv_word_1 = model_sg.get_word_vector(word_1)
                wv_word_2 = model_sg.get_word_vector(word_2)
                wv_word_3 = model_sg.get_word_vector(word_3)
                wv_calculated = wv_word_2 - wv_word_1 + wv_word_3
                word_predicted_curr = max(list_of_words, key=lambda word: cosine_similarity([model_sg.get_word_vector(word)], [wv_calculated]))
                

            except:
                continue
            if word_predicted_curr == word_4:
                true_predicted += 1
            else:
                false_predicted += 1

print("true_predicted", true_predicted)
print("false_predicted", false_predicted)
# print("true_predicted", true_predicted, true_predicted / (true_predicted+false_predicted))
# print("false_predicted", false_predicted, false_predicted / (true_predicted+false_predicted))


In [6]:
# Baca file word_analogy untuk evaluasi word embedding dengan teknik word analogy (menggunakan library)

dir_path_word_analogy = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Word_analogy/Word_Analogy.txt'

true_predicted = 0
false_predicted = 0



with open(dir_path_word_analogy , 'r') as file:
    content = file.read()
    lines = content.splitlines()
    for line in lines:
        item_in_line = line.split(" ")
        if len(item_in_line) == 4: # untuk buang baris yang merupakan keterangan
            word_1 = item_in_line[0].lower()
            word_2 = item_in_line[1].lower()
            word_3 = item_in_line[2].lower()
            word_4 = item_in_line[3].lower()
            word_predicted_curr = ""
            try:
                list_of_result_analogy = model_sg.get_analogies(word_1, word_2, word_3)
                word_predicted_curr = list_of_result_analogy[0][1] 
                if word_predicted_curr == word_4:
                    true_predicted += 1
                else:
                    false_predicted += 1

            except:
                continue
                
print("true_predicted", true_predicted)
print("false_predicted", false_predicted)            

true_predicted 59
false_predicted 19485


In [27]:
# Membuat model word embedding fasttext menggunakan CBOW 5
model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='cbow', lr=0.01, dim=40, ws=5, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0


Progress:  19.0% words/sec/thread:   37037 lr:  0.008102 avg.loss:  1.206894 ETA:   3h25m25s  0.0% words/sec/thread:   54026 lr:  0.009999 avg.loss:  3.711543 ETA:   2h53m48s   56069 lr:  0.009995 avg.loss:  2.788934 ETA:   2h47m25s  0.1% words/sec/thread:   55957 lr:  0.009986 avg.loss:  2.463470 ETA:   2h47m35s  0.3% words/sec/thread:   47317 lr:  0.009969 avg.loss:  2.301533 ETA:   3h17m51sh33m42s  0.6% words/sec/thread:   44100 lr:  0.009942 avg.loss:  2.166950 ETA:   3h31m43s0.7% words/sec/thread:   44362 lr:  0.009926 avg.loss:  2.107216 ETA:   3h30m 7s avg.loss:  2.058970 ETA:   3h22m33s  0.9% words/sec/thread:   45955 lr:  0.009908 avg.loss:  2.051283 ETA:   3h22m28s  1.6% words/sec/thread:   46892 lr:  0.009840 avg.loss:  1.954723 ETA:   3h17m 4s42ss  1.9% words/sec/thread:   46492 lr:  0.009807 avg.loss:  1.916842 ETA:   3h18m 6s  2.0% words/sec/thread:   46515 lr:  0.009803 avg.loss:  1.913066 ETA:   3h17m55s  2.0% words/sec/thread:   46496 lr:  0.009802 avg.loss:  1.911902 

Progress:  61.7% words/sec/thread:   53632 lr:  0.003832 avg.loss:  0.596463 ETA:   1h 7m 6s 55.3% words/sec/thread:   52476 lr:  0.004473 avg.loss:  0.627356 ETA:   1h20m 3s2s0.626882 ETA:   1h19m50s  1h19m45s 55.5% words/sec/thread:   52530 lr:  0.004454 avg.loss:  0.626411 ETA:   1h19m37s55.5% words/sec/thread:   52553 lr:  0.004448 avg.loss:  0.626157 ETA:   1h19m29s 55.5% words/sec/thread:   52563 lr:  0.004446 avg.loss:  0.626034 ETA:   1h19m25s 55.7% words/sec/thread:   52623 lr:  0.004431 avg.loss:  0.625155 ETA:   1h19m 4s% words/sec/thread:   52651 lr:  0.004424 avg.loss:  0.624701 ETA:   1h18m54s 55.8% words/sec/thread:   52659 lr:  0.004420 avg.loss:  0.624493 ETA:   1h18m49s 55.8% words/sec/thread:   52661 lr:  0.004418 avg.loss:  0.624364 ETA:   1h18m47savg.loss:  0.624269 ETA:   1h18m46sm44s 55.9% words/sec/thread:   52650 lr:  0.004408 avg.loss:  0.623813 ETA:   1h18m37s 56.0% words/sec/thread:   52647 lr:  0.004404 avg.loss:  0.623620 ETA:   1h18m33s 0.004399 avg.loss:

Progress:  67.2% words/sec/thread:   54564 lr:  0.003281 avg.loss:  0.572064 ETA:   0h56m28s 61.7% words/sec/thread:   53635 lr:  0.003829 avg.loss:  0.596281 ETA:   1h 7m 2s 61.7% words/sec/thread:   53643 lr:  0.003826 avg.loss:  0.596136 ETA:   1h 6m58s 61.9% words/sec/thread:   53669 lr:  0.003814 avg.loss:  0.595557 ETA:   1h 6m44s 61.9% words/sec/thread:   53684 lr:  0.003807 avg.loss:  0.595273 ETA:   1h 6m36s 62.0% words/sec/thread:   53690 lr:  0.003804 avg.loss:  0.595155 ETA:   1h 6m32s 62.0% words/sec/thread:   53701 lr:  0.003800 avg.loss:  0.594984 ETA:   1h 6m27s 62.0% words/sec/thread:   53707 lr:  0.003798 avg.loss:  0.594892 ETA:   1h 6m24s 0.003792 avg.loss:  0.594675 ETA:   1h 6m18s0.594606 ETA:   1h 6m16s words/sec/thread:   53700 lr:  0.003786 avg.loss:  0.594443 ETA:   1h 6m12s 62.2% words/sec/thread:   53705 lr:  0.003780 avg.loss:  0.594206 ETA:   1h 6m 6s 62.2% words/sec/thread:   53708 lr:  0.003777 avg.loss:  0.594085 ETA:   1h 6m 2s 62.3% words/sec/thread: 

Progress:  73.5% words/sec/thread:   55795 lr:  0.002645 avg.loss:  0.546615 ETA:   0h44m31s 67.2% words/sec/thread:   54570 lr:  0.003279 avg.loss:  0.571986 ETA:   0h56m25s avg.loss:  0.571598 ETA:   0h56m15s 0.003261 avg.loss:  0.571233 ETA:   0h56m 5s 67.4% words/sec/thread:   54606 lr:  0.003258 avg.loss:  0.571127 ETA:   0h56m 2s 67.5% words/sec/thread:   54619 lr:  0.003253 avg.loss:  0.570914 ETA:   0h55m56s 67.5% words/sec/thread:   54630 lr:  0.003249 avg.loss:  0.570724 ETA:   0h55m50s 67.5% words/sec/thread:   54636 lr:  0.003246 avg.loss:  0.570592 ETA:   0h55m47s 67.6% words/sec/thread:   54642 lr:  0.003243 avg.loss:  0.570437 ETA:   0h55m43s 67.6% words/sec/thread:   54649 lr:  0.003238 avg.loss:  0.570225 ETA:   0h55m38s 67.6% words/sec/thread:   54653 lr:  0.003236 avg.loss:  0.570127 ETA:   0h55m36s 67.7% words/sec/thread:   54656 lr:  0.003234 avg.loss:  0.570030 ETA:   0h55m34s 67.7% words/sec/thread:   54655 lr:  0.003229 avg.loss:  0.569746 ETA:   0h55m28s% words

Progress:  86.3% words/sec/thread:   60474 lr:  0.001368 avg.loss:  0.497114 ETA:   0h21m14s 73.6% words/sec/thread:   55800 lr:  0.002643 avg.loss:  0.546535 ETA:   0h44m29s 73.6% words/sec/thread:   55800 lr:  0.002643 avg.loss:  0.546532 ETA:   0h44m29s 73.6% words/sec/thread:   55817 lr:  0.002636 avg.loss:  0.546243 ETA:   0h44m21s 73.7% words/sec/thread:   55825 lr:  0.002633 avg.loss:  0.546117 ETA:   0h44m17s 73.7% words/sec/thread:   55834 lr:  0.002630 avg.loss:  0.545995 ETA:   0h44m14s 73.9% words/sec/thread:   55899 lr:  0.002612 avg.loss:  0.545276 ETA:   0h43m53s 74.0% words/sec/thread:   55933 lr:  0.002603 avg.loss:  0.544954 ETA:   0h43m42s 74.2% words/sec/thread:   56017 lr:  0.002580 avg.loss:  0.544173 ETA:   0h43m15s 74.2% words/sec/thread:   56038 lr:  0.002575 avg.loss:  0.543995 ETA:   0h43m 9s0.543977 ETA:   0h43m 9s/sec/thread:   56014 lr:  0.002574 avg.loss:  0.543955 ETA:   0h43m 9s  0h42m25s% words/sec/thread:   56188 lr:  0.002523 avg.loss:  0.541957 ETA:

Progress:  95.5% words/sec/thread:   62683 lr:  0.000454 avg.loss:  0.451506 ETA:   0h 6m47s 86.4% words/sec/thread:   60507 lr:  0.001358 avg.loss:  0.496594 ETA:   0h21m 4s86.4% words/sec/thread:   60507 lr:  0.001358 avg.loss:  0.496582 ETA:   0h21m 4s 86.6% words/sec/thread:   60559 lr:  0.001344 avg.loss:  0.495838 ETA:   0h20m50s  60574 lr:  0.001339 avg.loss:  0.495568 ETA:   0h20m45s 86.7% words/sec/thread:   60587 lr:  0.001335 avg.loss:  0.495338 ETA:   0h20m41s lr:  0.001332 avg.loss:  0.495168 ETA:   0h20m38s words/sec/thread:   60615 lr:  0.001325 avg.loss:  0.494825 ETA:   0h20m32s words/sec/thread:   60662 lr:  0.001309 avg.loss:  0.493954 ETA:   0h20m16s  60659 lr:  0.001308 avg.loss:  0.493861 ETA:   0h20m14s words/sec/thread:   60676 lr:  0.001302 avg.loss:  0.493543 ETA:   0h20m 8s 87.0% words/sec/thread:   60684 lr:  0.001298 avg.loss:  0.493357 ETA:   0h20m 5s   60696 lr:  0.001293 avg.loss:  0.493073 ETA:   0h20m 0s  0.001290 avg.loss:  0.492884 ETA:   0h19m57s 87

Progress: 100.0% words/sec/thread:   62646 lr:  0.000000 avg.loss:  0.431674 ETA:   0h 0m 0s451168 ETA:   0h 6m41s.000440 avg.loss:  0.450872 ETA:   0h 6m35s000425 avg.loss:  0.450183 ETA:   0h 6m22sthread:   62592 lr:  0.000424 avg.loss:  0.450128 ETA:   0h 6m21s00416 avg.loss:  0.449775 ETA:   0h 6m14s406 avg.loss:  0.449333 ETA:   0h 6m 5sETA:   0h 6m 1sad:   62595 lr:  0.000399 avg.loss:  0.449019 ETA:   0h 5m59s:   62620 lr:  0.000379 avg.loss:  0.448103 ETA:   0h 5m40s 0h 5m23s 96.5% words/sec/thread:   62627 lr:  0.000349 avg.loss:  0.446785 ETA:   0h 5m14s 96.5% words/sec/thread:   62624 lr:  0.000347 avg.loss:  0.446696 ETA:   0h 5m12s 96.6% words/sec/thread:   62617 lr:  0.000342 avg.loss:  0.446480 ETA:   0h 5m 8s 96.6% words/sec/thread:   62615 lr:  0.000341 avg.loss:  0.446420 ETA:   0h 5m 6s 96.6% words/sec/thread:   62620 lr:  0.000338 avg.loss:  0.446292 ETA:   0h 5m 4s 96.7% words/sec/thread:   62626 lr:  0.000334 avg.loss:  0.446097 ETA:   0h 5m 0s 96.8% words/sec/thr

In [28]:
# save model
model_cbow.save_model("model_cbow_fasttext_5.bin")

In [29]:
# load model
model_cbow = fasttext.load_model("model_cbow_fasttext_5.bin")

In [30]:
# relatedness model cbow 5
print(relatedness(model_cbow))

0.1327185218795432


In [31]:
# word analogy model cbow 5
word_analogy(model_cbow)

true_predicted 286 0.014633647155137127
false_predicted 19258 0.9853663528448628


In [32]:
# Membuat model word embedding fasttext menggunakan skip gram 5
model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='skipgram', lr=0.01, dim=40, ws=5, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0


Progress:   6.3% words/sec/thread:   20471 lr:  0.009366 avg.loss:  0.745761 ETA:   7h 9m41s  0.0% words/sec/thread:   18093 lr:  0.009999 avg.loss:  3.201104 ETA:   8h39m 0s  23387 lr:  0.009988 avg.loss:  2.093537 ETA:   6h41m 4s0.009988 avg.loss:  2.084861 ETA:   6h37m18sh37m40srds/sec/thread:   22123 lr:  0.009980 avg.loss:  2.058969 ETA:   7h 3m36slr:  0.009978 avg.loss:  2.058563 ETA:   7h 2m41sds/sec/thread:   22137 lr:  0.009978 avg.loss:  2.058720 ETA:   7h 3m16s 2.059032 ETA:   7h 1m26sr:  0.009976 avg.loss:  2.058286 ETA:   7h 2m15sgress:   0.3% words/sec/thread:   21570 lr:  0.009968 avg.loss:  2.052738 ETA:   7h13m58s962 avg.loss:  2.046649 ETA:   7h12m22sETA:   7h10m 1sh40m17s  0.6% words/sec/thread:   19664 lr:  0.009942 avg.loss:  2.051588 ETA:   7h54m49s  19193 lr:  0.009938 avg.loss:  2.053125 ETA:   8h 6m15s  0.7% words/sec/thread:   18939 lr:  0.009935 avg.loss:  2.053823 ETA:   8h12m36s 0.009931 avg.loss:  2.054352 ETA:   8h24m31s  0.7% words/sec/thread:   17869 lr

Progress:  38.3% words/sec/thread:   20641 lr:  0.006168 avg.loss:  0.189964 ETA:   4h40m38s 29.2% words/sec/thread:   21715 lr:  0.007082 avg.loss:  0.246948 ETA:   5h 6m15s 29.2% words/sec/thread:   21711 lr:  0.007080 avg.loss:  0.246799 ETA:   5h 6m14s 29.2% words/sec/thread:   21712 lr:  0.007078 avg.loss:  0.246638 ETA:   5h 6m 8s  5h 6m 3s 29.2% words/sec/thread:   21715 lr:  0.007076 avg.loss:  0.246465 ETA:   5h 6m 0ss 29.4% words/sec/thread:   21661 lr:  0.007057 avg.loss:  0.244937 ETA:   5h 5m57s 29.5% words/sec/thread:   21647 lr:  0.007048 avg.loss:  0.244268 ETA:   5h 5m46s 29.6% words/sec/thread:   21636 lr:  0.007042 avg.loss:  0.243768 ETA:   5h 5m39s 0.243344 ETA:   5h 5m26s 29.7% words/sec/thread:   21631 lr:  0.007031 avg.loss:  0.242860 ETA:   5h 5m13s 29.8% words/sec/thread:   21633 lr:  0.007018 avg.loss:  0.241860 ETA:   5h 4m39s 29.9% words/sec/thread:   21609 lr:  0.007009 avg.loss:  0.241173 ETA:   5h 4m36s30.0% words/sec/thread:   21571 lr:  0.006999 avg.lo

Progress:  46.4% words/sec/thread:   19629 lr:  0.005360 avg.loss:  0.158055 ETA:   4h16m26s 38.3% words/sec/thread:   20638 lr:  0.006167 avg.loss:  0.189898 ETA:   4h40m36s38.3% words/sec/thread:   20638 lr:  0.006166 avg.loss:  0.189870 ETA:   4h40m35s 38.6% words/sec/thread:   20600 lr:  0.006144 avg.loss:  0.188797 ETA:   4h40m 5s 38.6% words/sec/thread:   20596 lr:  0.006142 avg.loss:  0.188716 ETA:   4h40m 3s 38.6% words/sec/thread:   20591 lr:  0.006139 avg.loss:  0.188589 ETA:   4h39m59s 38.6% words/sec/thread:   20586 lr:  0.006135 avg.loss:  0.188384 ETA:   4h39m52s 38.7% words/sec/thread:   20573 lr:  0.006128 avg.loss:  0.188052 ETA:   4h39m43s38.8% words/sec/thread:   20561 lr:  0.006119 avg.loss:  0.187643 ETA:   4h39m29s 38.9% words/sec/thread:   20548 lr:  0.006110 avg.loss:  0.187209 ETA:   4h39m14s words/sec/thread:   20542 lr:  0.006106 avg.loss:  0.187021 ETA:   4h39m 8s 39.1% words/sec/thread:   20504 lr:  0.006085 avg.loss:  0.186064 ETA:   4h38m41s 39.2% words/s

Progress:  54.2% words/sec/thread:   18909 lr:  0.004577 avg.loss:  0.136246 ETA:   3h47m20s 46.4% words/sec/thread:   19629 lr:  0.005359 avg.loss:  0.158014 ETA:   4h16m23s 46.4% words/sec/thread:   19626 lr:  0.005356 avg.loss:  0.157925 ETA:   4h16m17s% words/sec/thread:   19623 lr:  0.005354 avg.loss:  0.157855 ETA:   4h16m13s 46.5% words/sec/thread:   19622 lr:  0.005353 avg.loss:  0.157821 ETA:   4h16m11s 46.5% words/sec/thread:   19621 lr:  0.005352 avg.loss:  0.157787 ETA:   4h16m 8s0.005341 avg.loss:  0.157451 ETA:   4h15m44s 46.6% words/sec/thread:   19613 lr:  0.005340 avg.loss:  0.157389 ETA:   4h15m39sh15m 2s35s46.9% words/sec/thread:   19584 lr:  0.005308 avg.loss:  0.156364 ETA:   4h14m30s47.0% words/sec/thread:   19581 lr:  0.005304 avg.loss:  0.156257 ETA:   4h14m23s 0.005302 avg.loss:  0.156196 ETA:   4h14m20s4h14m 0s13m37s% words/sec/thread:   19546 lr:  0.005267 avg.loss:  0.155077 ETA:   4h13m 2s47.3% words/sec/thread:   19545 lr:  0.005265 avg.loss:  0.155026 ETA

...

Progress:  73.1% words/sec/thread:   18078 lr:  0.002691 avg.loss:  0.102906 ETA:   2h19m47sm53s 64.2% words/sec/thread:   18390 lr:  0.003579 avg.loss:  0.116182 ETA:   3h 2m46s 64.2% words/sec/thread:   18388 lr:  0.003576 avg.loss:  0.116126 ETA:   3h 2m37s 64.3% words/sec/thread:   18384 lr:  0.003567 avg.loss:  0.115968 ETA:   3h 2m11s 64.4% words/sec/thread:   18379 lr:  0.003557 avg.loss:  0.115803 ETA:   3h 1m44s 0.003556 avg.loss:  0.115790 ETA:   3h 1m42s 64.5% words/sec/thread:   18378 lr:  0.003550 avg.loss:  0.115686 ETA:   3h 1m24s4s  18371 lr:  0.003529 avg.loss:  0.115326 ETA:   3h 0m23s 64.9% words/sec/thread:   18362 lr:  0.003508 avg.loss:  0.114983 ETA:   2h59m25sm21s  18360 lr:  0.003504 avg.loss:  0.114919 ETA:   2h59m14s 65.0% words/sec/thread:   18356 lr:  0.003498 avg.loss:  0.114807 ETA:   2h58m56s 65.3% words/sec/thread:   18344 lr:  0.003468 avg.loss:  0.114320 ETA:   2h57m32s 65.3% words/sec/thread:   18345 lr:  0.003466 avg.loss:  0.114284 ETA:   2h57m25s 

Progress:  80.8% words/sec/thread:   17754 lr:  0.001922 avg.loss:  0.093839 ETA:   1h41m40s44s 73.1% words/sec/thread:   18076 lr:  0.002690 avg.loss:  0.102889 ETA:   2h19m44s 73.3% words/sec/thread:   18073 lr:  0.002673 avg.loss:  0.102675 ETA:   2h18m54s 73.3% words/sec/thread:   18072 lr:  0.002673 avg.loss:  0.102669 ETA:   2h18m53s73.4% words/sec/thread:   18069 lr:  0.002661 avg.loss:  0.102508 ETA:   2h18m17s 73.4% words/sec/thread:   18069 lr:  0.002658 avg.loss:  0.102466 ETA:   2h18m 7ss  18067 lr:  0.002655 avg.loss:  0.102427 ETA:   2h17m58s 73.5% words/sec/thread:   18065 lr:  0.002654 avg.loss:  0.102426 ETA:   2h17m59s 73.5% words/sec/thread:   18048 lr:  0.002646 avg.loss:  0.102309 ETA:   2h17m39s 73.6% words/sec/thread:   18046 lr:  0.002641 avg.loss:  0.102254 ETA:   2h17m26s 73.6% words/sec/thread:   18045 lr:  0.002636 avg.loss:  0.102184 ETA:   2h17m10s 73.7% words/sec/thread:   18042 lr:  0.002628 avg.loss:  0.102081 ETA:   2h16m46s 73.9% words/sec/thread:   1

Progress:  89.0% words/sec/thread:   17521 lr:  0.001096 avg.loss:  0.085818 ETA:   0h58m45s 80.8% words/sec/thread:   17753 lr:  0.001918 avg.loss:  0.093790 ETA:   1h41m26s avg.loss:  0.093745 ETA:   1h41m13s 80.9% words/sec/thread:   17750 lr:  0.001906 avg.loss:  0.093671 ETA:   1h40m51s 81.0% words/sec/thread:   17746 lr:  0.001897 avg.loss:  0.093573 ETA:   1h40m23s 81.2% words/sec/thread:   17745 lr:  0.001878 avg.loss:  0.093371 ETA:   1h39m23s 81.4% words/sec/thread:   17740 lr:  0.001863 avg.loss:  0.093205 ETA:   1h38m36ss 81.5% words/sec/thread:   17735 lr:  0.001845 avg.loss:  0.093021 ETA:   1h37m42s81.6% words/sec/thread:   17734 lr:  0.001844 avg.loss:  0.093011 ETA:   1h37m39s words/sec/thread:   17734 lr:  0.001841 avg.loss:  0.092974 ETA:   1h37m28s 81.6% words/sec/thread:   17734 lr:  0.001840 avg.loss:  0.092970 ETA:   1h37m27s 81.6% words/sec/thread:   17734 lr:  0.001839 avg.loss:  0.092951 ETA:   1h37m21s 81.7% words/sec/thread:   17734 lr:  0.001833 avg.loss:  

Progress:  97.2% words/sec/thread:   17344 lr:  0.000283 avg.loss:  0.079219 ETA:   0h15m18s% words/sec/thread:   17511 lr:  0.001062 avg.loss:  0.085514 ETA:   0h56m57s 89.4% words/sec/thread:   17509 lr:  0.001057 avg.loss:  0.085471 ETA:   0h56m41ss 89.6% words/sec/thread:   17507 lr:  0.001044 avg.loss:  0.085353 ETA:   0h55m58s89.6% words/sec/thread:   17507 lr:  0.001042 avg.loss:  0.085337 ETA:   0h55m52sm43s 89.7% words/sec/thread:   17504 lr:  0.001034 avg.loss:  0.085273 ETA:   0h55m29s 89.7% words/sec/thread:   17503 lr:  0.001026 avg.loss:  0.085203 ETA:   0h55m 4s0h55m 0s 89.8% words/sec/thread:   17502 lr:  0.001022 avg.loss:  0.085166 ETA:   0h54m50s 89.8% words/sec/thread:   17501 lr:  0.001017 avg.loss:  0.085125 ETA:   0h54m35s 89.9% words/sec/thread:   17500 lr:  0.001009 avg.loss:  0.085048 ETA:   0h54m 7s 89.9% words/sec/thread:   17500 lr:  0.001008 avg.loss:  0.085046 ETA:   0h54m 6s 90.1% words/sec/thread:   17496 lr:  0.000992 avg.loss:  0.084901 ETA:   0h53m13

Progress: 100.0% words/sec/thread:   17306 lr:  0.000000 avg.loss:  0.077176 ETA:   0h 0m 0s avg.loss:  0.079172 ETA:   0h14m58s 97.4% words/sec/thread:   17342 lr:  0.000263 avg.loss:  0.079070 ETA:   0h14m13s 97.5% words/sec/thread:   17340 lr:  0.000250 avg.loss:  0.078975 ETA:   0h13m31s97.5% words/sec/thread:   17340 lr:  0.000249 avg.loss:  0.078970 ETA:   0h13m29s 97.5% words/sec/thread:   17339 lr:  0.000247 avg.loss:  0.078955 ETA:   0h13m22sss 97.7% words/sec/thread:   17336 lr:  0.000231 avg.loss:  0.078838 ETA:   0h12m30s12m22s 97.8% words/sec/thread:   17334 lr:  0.000220 avg.loss:  0.078756 ETA:   0h11m54s97.8% words/sec/thread:   17333 lr:  0.000215 avg.loss:  0.078722 ETA:   0h11m38s 97.9% words/sec/thread:   17333 lr:  0.000215 avg.loss:  0.078718 ETA:   0h11m37s 0.000210 avg.loss:  0.078682 ETA:   0h11m21s0h11m15s 98.2% words/sec/thread:   17331 lr:  0.000180 avg.loss:  0.078463 ETA:   0h 9m45s17332 lr:  0.000178 avg.loss:  0.078449 ETA:   0h 9m39s98.3% words/sec/thre

In [33]:
# save model
model_sg.save_model("model_sg_fasttext_5.bin")

In [34]:
# load model
model_sg = fasttext.load_model("model_sg_fasttext_5.bin")

In [35]:
# relatedness model sg 5
print(relatedness(model_sg))

0.26291515675221333


In [36]:
# word analogy model sg 5
word_analogy(model_sg)

true_predicted 136 0.006958657388456815
false_predicted 19408 0.9930413426115432


In [37]:
# Membuat model word embedding fasttext menggunakan CBOW 6
model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='cbow', lr=0.01, dim=10, ws=5, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0


Progress:  23.6% words/sec/thread:   50664 lr:  0.007644 avg.loss:  1.039778 ETA:   2h21m41s  48892 lr:  0.009999 avg.loss:  3.841030 ETA:   3h12m 3sg.loss:  2.755984 ETA:   2h38m 2s60325 lr:  0.009989 avg.loss:  2.522943 ETA:   2h35m30s2h31m38soss:  2.383193 ETA:   2h26m 5sss:  2.342800 ETA:   2h27m 8sm42s46ss.210906 ETA:   2h26m17sgress:   0.7% words/sec/thread:   63743 lr:  0.009927 avg.loss:  2.168132 ETA:   2h26m14s850 avg.loss:  2.083419 ETA:   2h23m11sETA:   2h22m48s  1.6% words/sec/thread:   65110 lr:  0.009837 avg.loss:  2.075922 ETA:   2h21m53s lr:  0.009827 avg.loss:  2.069562 ETA:   2h21m10s  2h21m 0s  1.8% words/sec/thread:   65569 lr:  0.009823 avg.loss:  2.065874 ETA:   2h20m41s  1.8% words/sec/thread:   65539 lr:  0.009820 avg.loss:  2.063413 ETA:   2h20m42s  1.8% words/sec/thread:   65579 lr:  0.009818 avg.loss:  2.062381 ETA:   2h20m35s  1.9% words/sec/thread:   65390 lr:  0.009814 avg.loss:  2.059932 ETA:   2h20m56sm22ssec/thread:   65579 lr:  0.009797 avg.loss:  2.0

In [38]:
# save model
model_cbow.save_model("model_cbow_fasttext_6.bin")

In [39]:
# load model
model_cbow = fasttext.load_model("model_cbow_fasttext_6.bin")

In [40]:
# relatedness model cbow 6
print(relatedness(model_cbow))

0.17455237543557034


In [41]:
# word analogy model cbow 6
word_analogy(model_cbow)

true_predicted 17 0.0008698321735571019
false_predicted 19527 0.9991301678264429


In [47]:
# Membuat model word embedding fasttext menggunakan skip gram 6
model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='skipgram', lr=0.01, dim=10, ws=5, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0


Progress:  12.3% words/sec/thread:   22505 lr:  0.008773 avg.loss:  0.558225 ETA:   6h 6m 4s  0.0% words/sec/thread:   25964 lr:  0.010000 avg.loss:  4.123903 ETA:   6h 1m41s0.0% words/sec/thread:   22788 lr:  0.009999 avg.loss:  2.947371 ETA:   6h52m 3s  0.1% words/sec/thread:   21648 lr:  0.009990 avg.loss:  2.095274 ETA:   7h13m22s0.1% words/sec/thread:   22248 lr:  0.009988 avg.loss:  2.080635 ETA:   7h 1m35s% words/sec/thread:   22536 lr:  0.009987 avg.loss:  2.078976 ETA:   6h56m10ss  0.2% words/sec/thread:   22749 lr:  0.009978 avg.loss:  2.062647 ETA:   6h51m52s  0.3% words/sec/thread:   22255 lr:  0.009974 avg.loss:  2.065646 ETA:   7h 0m53s  0.5% words/sec/thread:   22586 lr:  0.009952 avg.loss:  2.070419 ETA:   6h53m47s  0.5% words/sec/thread:   22580 lr:  0.009952 avg.loss:  2.070324 ETA:   6h53m53s3s  0.6% words/sec/thread:   22545 lr:  0.009943 avg.loss:  2.079706 ETA:   6h54m12s  0.6% words/sec/thread:   22594 lr:  0.009940 avg.loss:  2.083136 ETA:   6h53m 9s0.6% words/s

Progress:  63.8% words/sec/thread:   22400 lr:  0.003621 avg.loss:  0.116829 ETA:   2h31m49s 51.6% words/sec/thread:   22360 lr:  0.004841 avg.loss:  0.142864 ETA:   3h23m17s% words/sec/thread:   22364 lr:  0.004805 avg.loss:  0.141937 ETA:   3h21m47s 52.1% words/sec/thread:   22366 lr:  0.004786 avg.loss:  0.141432 ETA:   3h20m57sm47s% words/sec/thread:   22362 lr:  0.004778 avg.loss:  0.141217 ETA:   3h20m38s 52.5% words/sec/thread:   22365 lr:  0.004754 avg.loss:  0.140610 ETA:   3h19m37s34s 52.5% words/sec/thread:   22365 lr:  0.004752 avg.loss:  0.140551 ETA:   3h19m31s 52.8% words/sec/thread:   22370 lr:  0.004718 avg.loss:  0.139685 ETA:   3h18m 3s52.9% words/sec/thread:   22369 lr:  0.004706 avg.loss:  0.139373 ETA:   3h17m33s% words/sec/thread:   22370 lr:  0.004705 avg.loss:  0.139365 ETA:   3h17m32s 53.0% words/sec/thread:   22370 lr:  0.004702 avg.loss:  0.139280 ETA:   3h17m23ss16m38s  22376 lr:  0.004668 avg.loss:  0.138437 ETA:   3h15m55s15m26s 53.4% words/sec/thread:   

Progress:  80.8% words/sec/thread:   24175 lr:  0.001922 avg.loss:  0.093622 ETA:   1h14m39s 63.8% words/sec/thread:   22401 lr:  0.003620 avg.loss:  0.116802 ETA:   2h31m45s  22400 lr:  0.003570 avg.loss:  0.115942 ETA:   2h29m40s 64.4% words/sec/thread:   22401 lr:  0.003565 avg.loss:  0.115854 ETA:   2h29m27s 64.4% words/sec/thread:   22400 lr:  0.003562 avg.loss:  0.115807 ETA:   2h29m20s5sh28m47s 64.7% words/sec/thread:   22403 lr:  0.003535 avg.loss:  0.115352 ETA:   2h28m10s 64.8% words/sec/thread:   22401 lr:  0.003517 avg.loss:  0.115052 ETA:   2h27m26s 65.2% words/sec/thread:   22400 lr:  0.003482 avg.loss:  0.114458 ETA:   2h25m57s 65.2% words/sec/thread:   22399 lr:  0.003479 avg.loss:  0.114424 ETA:   2h25m52s 65.2% words/sec/thread:   22400 lr:  0.003478 avg.loss:  0.114399 ETA:   2h25m48ss 65.2% words/sec/thread:   22399 lr:  0.003477 avg.loss:  0.114377 ETA:   2h25m45s 65.3% words/sec/thread:   22400 lr:  0.003468 avg.loss:  0.114230 ETA:   2h25m22s 0.003431 avg.loss:  

Progress: 100.0% words/sec/thread:   26275 lr:  0.000000 avg.loss:  0.076787 ETA:   0h 0m 0s 80.9% words/sec/thread:   24191 lr:  0.001909 avg.loss:  0.093485 ETA:   1h14m 6sh13m28s 81.2% words/sec/thread:   24223 lr:  0.001885 avg.loss:  0.093224 ETA:   1h13m 4s lr:  0.001829 avg.loss:  0.092628 ETA:   1h10m41s% words/sec/thread:   24300 lr:  0.001826 avg.loss:  0.092601 ETA:   1h10m34s  24345 lr:  0.001789 avg.loss:  0.092207 ETA:   1h 9m 0s 82.4% words/sec/thread:   24382 lr:  0.001761 avg.loss:  0.091913 ETA:   1h 7m48s 82.6% words/sec/thread:   24411 lr:  0.001739 avg.loss:  0.091684 ETA:   1h 6m53s 82.8% words/sec/thread:   24431 lr:  0.001720 avg.loss:  0.091492 ETA:   1h 6m 6s52s 83.0% words/sec/thread:   24456 lr:  0.001697 avg.loss:  0.091257 ETA:   1h 5m 9s 83.1% words/sec/thread:   24470 lr:  0.001685 avg.loss:  0.091140 ETA:   1h 4m41s4m36s ETA:   1h 3m40s83.7% words/sec/thread:   24531 lr:  0.001634 avg.loss:  0.090621 ETA:   1h 2m33s% words/sec/thread:   24542 lr:  0.001

In [48]:
# save model
model_sg.save_model("model_sg_fasttext_6.bin")

In [49]:
# load model
model_sg = fasttext.load_model("model_sg_fasttext_6.bin")

In [50]:
# relatedness model sg 6
print(relatedness(model_sg))

0.19263002339117155


In [51]:
# word analogy model sg 6
word_analogy(model_sg)

true_predicted 3 0.0001534997953336062
false_predicted 19541 0.9998465002046664


In [52]:
# Membuat model word embedding fasttext menggunakan CBOW 7
model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='cbow', lr=0.01, dim=10, ws=7, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0


Progress:  48.6% words/sec/thread:  128760 lr:  0.005135 avg.loss:  0.683039 ETA:   0h37m27s 243725 lr:  0.009995 avg.loss:  2.698814 ETA:   0h38m30s  0.2% words/sec/thread:  256215 lr:  0.009983 avg.loss:  2.417382 ETA:   0h36m35s 0.009974 avg.loss:  2.332937 ETA:   0h37m51s  0.6% words/sec/thread:  218256 lr:  0.009943 avg.loss:  2.208432 ETA:   0h42m47s  0.9% words/sec/thread:  176433 lr:  0.009913 avg.loss:  2.148323 ETA:   0h52m45s  0.9% words/sec/thread:  157887 lr:  0.009907 avg.loss:  2.138327 ETA:   0h58m55s  1.2% words/sec/thread:  136778 lr:  0.009884 avg.loss:  2.113750 ETA:   1h 7m51s  1.2% words/sec/thread:  132591 lr:  0.009882 avg.loss:  2.112632 ETA:   1h 9m59s2.102285 ETA:   1h14m49s 121495 lr:  0.009863 avg.loss:  2.098529 ETA:   1h16m14s words/sec/thread:  118992 lr:  0.009860 avg.loss:  2.096909 ETA:   1h17m49s  1.5% words/sec/thread:  114942 lr:  0.009854 avg.loss:  2.094136 ETA:   1h20m30s  1.8% words/sec/thread:   95043 lr:  0.009825 avg.loss:  2.074800 ETA:   1

Progress: 100.0% words/sec/thread:  108810 lr:  0.000000 avg.loss:  0.453539 ETA:   0h 0m 0s 99.9% words/sec/thread:  108812 lr:  0.000012 avg.loss:  0.454072 ETA:   0h 0m 6s 99.9% words/sec/thread:  108815 lr:  0.000005 avg.loss:  0.453771 ETA:   0h 0m 2s  0h 0m 0s


In [53]:
# save model
model_cbow.save_model("model_cbow_fasttext_7.bin")

In [54]:
# load model
model_cbow = fasttext.load_model("model_cbow_fasttext_7.bin")

In [55]:
# relatedness model cbow 7
print(relatedness(model_cbow))

0.18112780398738887


In [56]:
# word analogy model cbow 7
word_analogy(model_cbow)

true_predicted 20 0.001023331968890708
false_predicted 19524 0.9989766680311093


In [57]:
# Membuat model word embedding fasttext menggunakan skip gram 7
model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='skipgram', lr=0.01, dim=10, ws=7, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0


Progress:   8.4% words/sec/thread:   30122 lr:  0.009164 avg.loss:  0.563777 ETA:   4h45m41s  0.009996 avg.loss:  2.040298 ETA:   2h10m26s  0.2% words/sec/thread:   85684 lr:  0.009984 avg.loss:  1.989413 ETA:   1h49m25s  0.2% words/sec/thread:   84713 lr:  0.009981 avg.loss:  1.989356 ETA:   1h50m39s  0.2% words/sec/thread:   83825 lr:  0.009977 avg.loss:  1.994571 ETA:   1h51m46s  0.3% words/sec/thread:   81243 lr:  0.009972 avg.loss:  2.000939 ETA:   1h55m16s  0.3% words/sec/thread:   78019 lr:  0.009969 avg.loss:  2.003054 ETA:   2h 0m 0s  76587 lr:  0.009965 avg.loss:  2.008452 ETA:   2h 2m11s  0.4% words/sec/thread:   74346 lr:  0.009961 avg.loss:  2.015935 ETA:   2h 5m49s  0.4% words/sec/thread:   68320 lr:  0.009956 avg.loss:  2.024936 ETA:   2h16m51s  0.5% words/sec/thread:   66735 lr:  0.009954 avg.loss:  2.028543 ETA:   2h20m 4s  0.5% words/sec/thread:   65074 lr:  0.009952 avg.loss:  2.032269 ETA:   2h23m37s avg.loss:  2.035031 ETA:   2h27m33s0.5% words/sec/thread:   61971 

Progress:  25.1% words/sec/thread:   31474 lr:  0.007493 avg.loss:  0.213121 ETA:   3h43m34s 20.0% words/sec/thread:   33891 lr:  0.007995 avg.loss:  0.264874 ETA:   3h41m32s20.1% words/sec/thread:   33891 lr:  0.007994 avg.loss:  0.264655 ETA:   3h41m30s  3h41m25s 20.1% words/sec/thread:   33890 lr:  0.007991 avg.loss:  0.264304 ETA:   3h41m25s 20.1% words/sec/thread:   33893 lr:  0.007988 avg.loss:  0.263984 ETA:   3h41m20s 20.1% words/sec/thread:   33881 lr:  0.007987 avg.loss:  0.263746 ETA:   3h41m22s 20.2% words/sec/thread:   33878 lr:  0.007983 avg.loss:  0.263303 ETA:   3h41m17s avg.loss:  0.262992 ETA:   3h41m12s 20.2% words/sec/thread:   33881 lr:  0.007981 avg.loss:  0.262976 ETA:   3h41m12s 20.2% words/sec/thread:   33872 lr:  0.007979 avg.loss:  0.262773 ETA:   3h41m13s 20.2% words/sec/thread:   33871 lr:  0.007975 avg.loss:  0.262337 ETA:   3h41m 7s 20.3% words/sec/thread:   33879 lr:  0.007972 avg.loss:  0.261874 ETA:   3h40m58s 20.3% words/sec/thread:   33885 lr:  0.007

Progress:  30.2% words/sec/thread:   29906 lr:  0.006984 avg.loss:  0.178183 ETA:   3h39m18s 25.1% words/sec/thread:   31449 lr:  0.007490 avg.loss:  0.212872 ETA:   3h43m39s 25.5% words/sec/thread:   31515 lr:  0.007453 avg.loss:  0.209850 ETA:   3h42m 5s 25.5% words/sec/thread:   31505 lr:  0.007451 avg.loss:  0.209750 ETA:   3h42m 6s% words/sec/thread:   31495 lr:  0.007450 avg.loss:  0.209644 ETA:   3h42m 8s31492 lr:  0.007449 avg.loss:  0.209601 ETA:   3h42m 8s 25.5% words/sec/thread:   31491 lr:  0.007448 avg.loss:  0.209509 ETA:   3h42m 7s 25.5% words/sec/thread:   31455 lr:  0.007446 avg.loss:  0.209346 ETA:   3h42m18s  3h42m10s words/sec/thread:   31445 lr:  0.007440 avg.loss:  0.208830 ETA:   3h42m11s 25.7% words/sec/thread:   31412 lr:  0.007428 avg.loss:  0.207900 ETA:   3h42m 4ss 25.8% words/sec/thread:   31394 lr:  0.007417 avg.loss:  0.207082 ETA:   3h41m52s 26.1% words/sec/thread:   31409 lr:  0.007394 avg.loss:  0.205316 ETA:   3h41m 5s 26.2% words/sec/thread:   31438 

Progress:  36.6% words/sec/thread:   29368 lr:  0.006337 avg.loss:  0.147767 ETA:   3h22m39s 30.2% words/sec/thread:   29903 lr:  0.006982 avg.loss:  0.178062 ETA:   3h39m15s30.2% words/sec/thread:   29903 lr:  0.006981 avg.loss:  0.178043 ETA:   3h39m14s 30.3% words/sec/thread:   29919 lr:  0.006972 avg.loss:  0.177520 ETA:   3h38m50s 30.3% words/sec/thread:   29922 lr:  0.006970 avg.loss:  0.177385 ETA:   3h38m45s 30.3% words/sec/thread:   29907 lr:  0.006965 avg.loss:  0.177136 ETA:   3h38m43s 30.4% words/sec/thread:   29912 lr:  0.006964 avg.loss:  0.177081 ETA:   3h38m39savg.loss:  0.176976 ETA:   3h38m33s 30.4% words/sec/thread:   29939 lr:  0.006958 avg.loss:  0.176708 ETA:   3h38m15s 30.5% words/sec/thread:   29963 lr:  0.006952 avg.loss:  0.176392 ETA:   3h37m54s 30.6% words/sec/thread:   29986 lr:  0.006945 avg.loss:  0.175949 ETA:   3h37m29s avg.loss:  0.175944 ETA:   3h37m29s 30.6% words/sec/thread:   29989 lr:  0.006942 avg.loss:  0.175831 ETA:   3h37m24s words/sec/thread:

...

Progress:  57.3% words/sec/thread:   29499 lr:  0.004267 avg.loss:  0.096377 ETA:   2h15m51ss% words/sec/thread:   29606 lr:  0.004904 avg.loss:  0.107733 ETA:   2h35m32s 51.0% words/sec/thread:   29611 lr:  0.004897 avg.loss:  0.107591 ETA:   2h35m17s 51.1% words/sec/thread:   29602 lr:  0.004888 avg.loss:  0.107426 ETA:   2h35m 4s% words/sec/thread:   29603 lr:  0.004878 avg.loss:  0.107212 ETA:   2h34m44s 51.2% words/sec/thread:   29602 lr:  0.004877 avg.loss:  0.107190 ETA:   2h34m42s 51.3% words/sec/thread:   29593 lr:  0.004871 avg.loss:  0.107071 ETA:   2h34m33s 51.3% words/sec/thread:   29584 lr:  0.004870 avg.loss:  0.107053 ETA:   2h34m34s 51.3% words/sec/thread:   29575 lr:  0.004868 avg.loss:  0.107029 ETA:   2h34m35s 51.3% words/sec/thread:   29573 lr:  0.004868 avg.loss:  0.107018 ETA:   2h34m34s 51.4% words/sec/thread:   29570 lr:  0.004860 avg.loss:  0.106866 ETA:   2h34m21s 51.6% words/sec/thread:   29594 lr:  0.004839 avg.loss:  0.106435 ETA:   2h33m32ss 51.7% words/s

Progress:  66.1% words/sec/thread:   30396 lr:  0.003386 avg.loss:  0.084213 ETA:   1h44m36s57.3% words/sec/thread:   29502 lr:  0.004265 avg.loss:  0.096343 ETA:   2h15m46s 57.4% words/sec/thread:   29506 lr:  0.004262 avg.loss:  0.096294 ETA:   2h15m39s 57.4% words/sec/thread:   29507 lr:  0.004261 avg.loss:  0.096279 ETA:   2h15m36sm25s 57.5% words/sec/thread:   29521 lr:  0.004253 avg.loss:  0.096157 ETA:   2h15m18s 57.5% words/sec/thread:   29525 lr:  0.004251 avg.loss:  0.096117 ETA:   2h15m12s  29554 lr:  0.004234 avg.loss:  0.095852 ETA:   2h14m32s 57.7% words/sec/thread:   29563 lr:  0.004228 avg.loss:  0.095754 ETA:   2h14m18s57.8% words/sec/thread:   29568 lr:  0.004225 avg.loss:  0.095700 ETA:   2h14m10s 57.8% words/sec/thread:   29572 lr:  0.004223 avg.loss:  0.095672 ETA:   2h14m 6s 57.9% words/sec/thread:   29589 lr:  0.004212 avg.loss:  0.095505 ETA:   2h13m41s 57.9% words/sec/thread:   29590 lr:  0.004210 avg.loss:  0.095478 ETA:   2h13m37s58.0% words/sec/thread:   295

Progress:  83.8% words/sec/thread:   32753 lr:  0.001625 avg.loss:  0.067689 ETA:   0h46m34s 66.2% words/sec/thread:   30401 lr:  0.003382 avg.loss:  0.084170 ETA:   1h44m29s% words/sec/thread:   30414 lr:  0.003374 avg.loss:  0.084063 ETA:   1h44m10s words/sec/thread:   30426 lr:  0.003365 avg.loss:  0.083962 ETA:   1h43m51s66.4% words/sec/thread:   30428 lr:  0.003364 avg.loss:  0.083942 ETA:   1h43m48s 66.5% words/sec/thread:   30443 lr:  0.003349 avg.loss:  0.083765 ETA:   1h43m17s 66.6% words/sec/thread:   30453 lr:  0.003341 avg.loss:  0.083673 ETA:   1h43m 1s 66.7% words/sec/thread:   30463 lr:  0.003329 avg.loss:  0.083537 ETA:   1h42m37s66.8% words/sec/thread:   30475 lr:  0.003318 avg.loss:  0.083414 ETA:   1h42m15s% words/sec/thread:   30481 lr:  0.003311 avg.loss:  0.083325 ETA:   1h41m59s 66.9% words/sec/thread:   30485 lr:  0.003307 avg.loss:  0.083288 ETA:   1h41m53s 67.1% words/sec/thread:   30502 lr:  0.003290 avg.loss:  0.083094 ETA:   1h41m18s 67.3% words/sec/thread:

Progress:  91.6% words/sec/thread:   32588 lr:  0.000841 avg.loss:  0.062391 ETA:   0h24m13s 83.8% words/sec/thread:   32752 lr:  0.001624 avg.loss:  0.067685 ETA:   0h46m33s 83.8% words/sec/thread:   32752 lr:  0.001624 avg.loss:  0.067683 ETA:   0h46m33s lr:  0.001623 avg.loss:  0.067679 ETA:   0h46m32s 83.8% words/sec/thread:   32753 lr:  0.001619 avg.loss:  0.067650 ETA:   0h46m25s 83.8% words/sec/thread:   32749 lr:  0.001618 avg.loss:  0.067641 ETA:   0h46m24s 83.8% words/sec/thread:   32742 lr:  0.001615 avg.loss:  0.067621 ETA:   0h46m20s 83.9% words/sec/thread:   32739 lr:  0.001614 avg.loss:  0.067611 ETA:   0h46m18savg.loss:  0.067606 ETA:   0h46m17s  32733 lr:  0.001611 avg.loss:  0.067588 ETA:   0h46m13s 32731 lr:  0.001609 avg.loss:  0.067573 ETA:   0h46m 9s9% words/sec/thread:   32729 lr:  0.001606 avg.loss:  0.067553 ETA:   0h46m 5s14 lr:  0.001603 avg.loss:  0.067527 ETA:   0h46m 0sords/sec/thread:   32705 lr:  0.001602 avg.loss:  0.067519 ETA:   0h45m59s lr:  0.001601

Progress: 100.0% words/sec/thread:   32812 lr:  0.000000 avg.loss:  0.057625 ETA:   0h 0m 0s 91.6% words/sec/thread:   32586 lr:  0.000839 avg.loss:  0.062380 ETA:   0h24m10s 0.062042 ETA:   0h22m33s 92.2% words/sec/thread:   32624 lr:  0.000780 avg.loss:  0.062023 ETA:   0h22m27s% words/sec/thread:   32630 lr:  0.000770 avg.loss:  0.061962 ETA:   0h22m10sm 6s 92.3% words/sec/thread:   32633 lr:  0.000767 avg.loss:  0.061942 ETA:   0h22m 4s 92.4% words/sec/thread:   32630 lr:  0.000756 avg.loss:  0.061873 ETA:   0h21m44s words/sec/thread:   32624 lr:  0.000736 avg.loss:  0.061757 ETA:   0h21m11s92.8% words/sec/thread:   32617 lr:  0.000715 avg.loss:  0.061628 ETA:   0h20m35s 92.9% words/sec/thread:   32617 lr:  0.000713 avg.loss:  0.061616 ETA:   0h20m31s 92.9% words/sec/thread:   32617 lr:  0.000710 avg.loss:  0.061600 ETA:   0h20m27s 92.9% words/sec/thread:   32616 lr:  0.000709 avg.loss:  0.061595 ETA:   0h20m25s92.9% words/sec/thread:   32615 lr:  0.000708 avg.loss:  0.061584 ETA: 

In [58]:
# save model
model_sg.save_model("model_sg_fasttext_7.bin")

In [59]:
# load model
model_sg = fasttext.load_model("model_sg_fasttext_7.bin")

In [60]:
# relatedness model sg 7
print(relatedness(model_sg))

0.19434465159254385


In [61]:
# word analogy model sg 7
word_analogy(model_sg)

true_predicted 6 0.0003069995906672124
false_predicted 19538 0.9996930004093327


In [62]:
# Membuat model word embedding fasttext menggunakan CBOW 8
model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='cbow', lr=0.01, dim=10, ws=9, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0


Progress:  35.3% words/sec/thread:   88372 lr:  0.006471 avg.loss:  0.827449 ETA:   1h 8m45s  0.1% words/sec/thread:  242667 lr:  0.009995 avg.loss:  2.659083 ETA:   0h38m40s  0.2% words/sec/thread:  237277 lr:  0.009982 avg.loss:  2.385128 ETA:   0h39m30s45m23s  0.4% words/sec/thread:  194024 lr:  0.009958 avg.loss:  2.240645 ETA:   0h48m12s 2.202187 ETA:   0h51m30s  0.6% words/sec/thread:  169857 lr:  0.009935 avg.loss:  2.180140 ETA:   0h54m55s2.172618 ETA:   0h57m29s  0.8% words/sec/thread:  146803 lr:  0.009921 avg.loss:  2.149299 ETA:   1h 3m27s  0.8% words/sec/thread:  143852 lr:  0.009917 avg.loss:  2.145303 ETA:   1h 4m44s  0.9% words/sec/thread:  141448 lr:  0.009915 avg.loss:  2.142191 ETA:   1h 5m49s lr:  0.009912 avg.loss:  2.138369 ETA:   1h 6m57s0.9% words/sec/thread:  135691 lr:  0.009908 avg.loss:  2.132564 ETA:   1h 8m34s  1.0% words/sec/thread:  131134 lr:  0.009903 avg.loss:  2.125693 ETA:   1h10m55s  1.0% words/sec/thread:  127850 lr:  0.009900 avg.loss:  2.121707 

Progress:  51.8% words/sec/thread:   88200 lr:  0.004816 avg.loss:  0.657728 ETA:   0h51m16s 41.7% words/sec/thread:   83984 lr:  0.005828 avg.loss:  0.745863 ETA:   1h 5m10s 41.8% words/sec/thread:   84012 lr:  0.005819 avg.loss:  0.744730 ETA:   1h 5m 2s ETA:   1h 4m59s 41.9% words/sec/thread:   84035 lr:  0.005812 avg.loss:  0.743920 ETA:   1h 4m56s84097 lr:  0.005798 avg.loss:  0.742533 ETA:   1h 4m45s 42.1% words/sec/thread:   84142 lr:  0.005791 avg.loss:  0.741794 ETA:   1h 4m38s 42.3% words/sec/thread:   84250 lr:  0.005774 avg.loss:  0.740005 ETA:   1h 4m21s 42.4% words/sec/thread:   84311 lr:  0.005761 avg.loss:  0.738573 ETA:   1h 4m 9s 42.5% words/sec/thread:   84388 lr:  0.005748 avg.loss:  0.737219 ETA:   1h 3m58s 42.6% words/sec/thread:   84437 lr:  0.005739 avg.loss:  0.736186 ETA:   1h 3m49s 42.7% words/sec/thread:   84445 lr:  0.005730 avg.loss:  0.735386 ETA:   1h 3m43s 42.8% words/sec/thread:   84475 lr:  0.005723 avg.loss:  0.734607 ETA:   1h 3m37s 42.9% words/sec/

Progress:  64.5% words/sec/thread:   92415 lr:  0.003550 avg.loss:  0.589187 ETA:   0h36m 4s 52.0% words/sec/thread:   88259 lr:  0.004804 avg.loss:  0.656916 ETA:   0h51m 6s 0.004794 avg.loss:  0.656289 ETA:   0h50m58s 52.1% words/sec/thread:   88316 lr:  0.004793 avg.loss:  0.656229 ETA:   0h50m57s 52.2% words/sec/thread:   88363 lr:  0.004783 avg.loss:  0.655646 ETA:   0h50m50s52.3% words/sec/thread:   88393 lr:  0.004775 avg.loss:  0.655163 ETA:   0h50m43s 52.3% words/sec/thread:   88424 lr:  0.004767 avg.loss:  0.654682 ETA:   0h50m37s 52.4% words/sec/thread:   88460 lr:  0.004756 avg.loss:  0.654057 ETA:   0h50m29s  0h50m24s 52.5% words/sec/thread:   88481 lr:  0.004746 avg.loss:  0.653359 ETA:   0h50m22s16s 52.6% words/sec/thread:   88507 lr:  0.004738 avg.loss:  0.652831 ETA:   0h50m16s 52.6% words/sec/thread:   88507 lr:  0.004735 avg.loss:  0.652664 ETA:   0h50m14s52.7% words/sec/thread:   88498 lr:  0.004731 avg.loss:  0.652393 ETA:   0h50m12s 52.8% words/sec/thread:   88533

Progress:  73.4% words/sec/thread:   87548 lr:  0.002662 avg.loss:  0.553436 ETA:   0h28m33s% words/sec/thread:   92473 lr:  0.003534 avg.loss:  0.588458 ETA:   0h35m53s92486 lr:  0.003530 avg.loss:  0.588290 ETA:   0h35m50s 64.8% words/sec/thread:   92517 lr:  0.003522 avg.loss:  0.587901 ETA:   0h35m44sh35m33s words/sec/thread:   92498 lr:  0.003501 avg.loss:  0.586951 ETA:   0h35m32s 65.0% words/sec/thread:   92476 lr:  0.003500 avg.loss:  0.586911 ETA:   0h35m32s 65.0% words/sec/thread:   92451 lr:  0.003495 avg.loss:  0.586732 ETA:   0h35m30s 65.1% words/sec/thread:   92438 lr:  0.003493 avg.loss:  0.586628 ETA:   0h35m29s 65.1% words/sec/thread:   92425 lr:  0.003487 avg.loss:  0.586363 ETA:   0h35m25s   0h35m19s.loss:  0.585268 ETA:   0h35m 8sloss:  0.584992 ETA:   0h35m 3s% words/sec/thread:   92459 lr:  0.003439 avg.loss:  0.584414 ETA:   0h34m55s 65.6% words/sec/thread:   92461 lr:  0.003435 avg.loss:  0.584256 ETA:   0h34m53s 65.7% words/sec/thread:   92465 lr:  0.003432 avg

Progress:  80.6% words/sec/thread:   83446 lr:  0.001944 avg.loss:  0.527810 ETA:   0h21m52s 0.002660 avg.loss:  0.553319 ETA:   0h28m31s 73.4% words/sec/thread:   87511 lr:  0.002657 avg.loss:  0.553203 ETA:   0h28m30s87456 lr:  0.002652 avg.loss:  0.552985 ETA:   0h28m28s5% words/sec/thread:   87445 lr:  0.002651 avg.loss:  0.552952 ETA:   0h28m28s73.6% words/sec/thread:   87381 lr:  0.002640 avg.loss:  0.552532 ETA:   0h28m22s 73.6% words/sec/thread:   87356 lr:  0.002636 avg.loss:  0.552354 ETA:   0h28m20s 73.7% words/sec/thread:   87343 lr:  0.002633 avg.loss:  0.552241 ETA:   0h28m18sh28m16s 73.8% words/sec/thread:   87284 lr:  0.002622 avg.loss:  0.551786 ETA:   0h28m12savg.loss:  0.551686 ETA:   0h28m11s 73.9% words/sec/thread:   87242 lr:  0.002614 avg.loss:  0.551465 ETA:   0h28m 8s 73.9% words/sec/thread:   87220 lr:  0.002611 avg.loss:  0.551320 ETA:   0h28m 6s 74.0% words/sec/thread:   87176 lr:  0.002605 avg.loss:  0.551066 ETA:   0h28m 3ss 74.0% words/sec/thread:   87140

Progress: 100.0% words/sec/thread:   83760 lr:  0.000000 avg.loss:  0.437445 ETA:   0h 0m 0s 80.7% words/sec/thread:   83382 lr:  0.001935 avg.loss:  0.527520 ETA:   0h21m47s80.7% words/sec/thread:   83306 lr:  0.001932 avg.loss:  0.527421 ETA:   0h21m46s0h21m44ss 80.8% words/sec/thread:   83270 lr:  0.001915 avg.loss:  0.526846 ETA:   0h21m36s 0.001915 avg.loss:  0.526818 ETA:   0h21m35s 81.1% words/sec/thread:   83254 lr:  0.001894 avg.loss:  0.526063 ETA:   0h21m22sh21m13s 81.4% words/sec/thread:   83328 lr:  0.001861 avg.loss:  0.524936 ETA:   0h20m58s 81.6% words/sec/thread:   83373 lr:  0.001840 avg.loss:  0.524259 ETA:   0h20m43s 82.3% words/sec/thread:   83575 lr:  0.001772 avg.loss:  0.522000 ETA:   0h19m54s 0.001703 avg.loss:  0.519841 ETA:   0h19m 9s 83.1% words/sec/thread:   83443 lr:  0.001688 avg.loss:  0.519275 ETA:   0h18m59s 83.2% words/sec/thread:   83446 lr:  0.001684 avg.loss:  0.519157 ETA:   0h18m57s  83445 lr:  0.001682 avg.loss:  0.519059 ETA:   0h18m55s ETA:   

In [63]:
# save model
model_cbow.save_model("model_cbow_fasttext_8.bin")

In [64]:
# load model
model_sg = fasttext.load_model("model_cbow_fasttext_8.bin")

In [65]:
# relatedness model cbow 8
print(relatedness(model_cbow))

0.19192062549985373


In [66]:
# word analogy model cbow 8
word_analogy(model_cbow)

true_predicted 18 0.0009209987720016373
false_predicted 19526 0.9990790012279983


In [67]:
# Membuat model word embedding fasttext menggunakan skip gram 8
model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='skipgram', lr=0.01, dim=10, ws=9, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0


Progress:   4.8% words/sec/thread:   40536 lr:  0.009519 avg.loss:  0.674594 ETA:   3h40m32s  0.0% words/sec/thread:   54611 lr:  0.009999 avg.loss:  2.259444 ETA:   2h51m56s  3h16m33s  0.1% words/sec/thread:   47393 lr:  0.009989 avg.loss:  1.926486 ETA:   3h17m55s0.1% words/sec/thread:   45317 lr:  0.009986 avg.loss:  1.931648 ETA:   3h26m56s avg.loss:  1.934190 ETA:   3h30m26s  0.2% words/sec/thread:   43479 lr:  0.009983 avg.loss:  1.932680 ETA:   3h35m37s  42983 lr:  0.009983 avg.loss:  1.932535 ETA:   3h38m 6s  0.2% words/sec/thread:   41541 lr:  0.009982 avg.loss:  1.933500 ETA:   3h45m39s  0.2% words/sec/thread:   40008 lr:  0.009980 avg.loss:  1.938598 ETA:   3h54m15s  0.2% words/sec/thread:   39500 lr:  0.009978 avg.loss:  1.943002 ETA:   3h57m13s  3h59m41s words/sec/thread:   37887 lr:  0.009973 avg.loss:  1.956249 ETA:   4h 7m11s  0.3% words/sec/thread:   35671 lr:  0.009969 avg.loss:  1.966003 ETA:   4h22m26s  0.3% words/sec/thread:   34897 lr:  0.009966 avg.loss:  1.97280

Progress:  43.2% words/sec/thread:   30595 lr:  0.005682 avg.loss:  0.103038 ETA:   2h54m23ss 35.0% words/sec/thread:   31860 lr:  0.006498 avg.loss:  0.125604 ETA:   3h11m31s 35.1% words/sec/thread:   31862 lr:  0.006494 avg.loss:  0.125464 ETA:   3h11m23s 35.1% words/sec/thread:   31858 lr:  0.006485 avg.loss:  0.125176 ETA:   3h11m10s10s 35.5% words/sec/thread:   31887 lr:  0.006451 avg.loss:  0.124019 ETA:   3h 9m59s 35.7% words/sec/thread:   31804 lr:  0.006433 avg.loss:  0.123424 ETA:   3h 9m57s  3h 9m56s 35.7% words/sec/thread:   31772 lr:  0.006426 avg.loss:  0.123199 ETA:   3h 9m56s 35.8% words/sec/thread:   31738 lr:  0.006422 avg.loss:  0.123043 ETA:   3h10m 0s 35.8% words/sec/thread:   31717 lr:  0.006415 avg.loss:  0.122843 ETA:   3h 9m57s35.9% words/sec/thread:   31707 lr:  0.006414 avg.loss:  0.122812 ETA:   3h 9m59s 35.9% words/sec/thread:   31688 lr:  0.006410 avg.loss:  0.122671 ETA:   3h 9m58s 36.0% words/sec/thread:   31661 lr:  0.006403 avg.loss:  0.122432 ETA:   3

Progress:  56.8% words/sec/thread:   30449 lr:  0.004316 avg.loss:  0.079692 ETA:   2h13m 7s 43.3% words/sec/thread:   30612 lr:  0.005671 avg.loss:  0.102789 ETA:   2h53m57s43.5% words/sec/thread:   30652 lr:  0.005650 avg.loss:  0.102330 ETA:   2h53m 6s 43.5% words/sec/thread:   30656 lr:  0.005648 avg.loss:  0.102276 ETA:   2h53m 0s36s 43.6% words/sec/thread:   30671 lr:  0.005635 avg.loss:  0.101998 ETA:   2h52m32sh52m19s 43.7% words/sec/thread:   30672 lr:  0.005627 avg.loss:  0.101811 ETA:   2h52m17s 43.8% words/sec/thread:   30678 lr:  0.005620 avg.loss:  0.101666 ETA:   2h52m 2s 44.2% words/sec/thread:   30724 lr:  0.005580 avg.loss:  0.100777 ETA:   2h50m33s 44.3% words/sec/thread:   30741 lr:  0.005568 avg.loss:  0.100520 ETA:   2h50m 5s49m15s54s 44.9% words/sec/thread:   30828 lr:  0.005508 avg.loss:  0.099248 ETA:   2h47m47s 45.0% words/sec/thread:   30845 lr:  0.005499 avg.loss:  0.099056 ETA:   2h47m25s 45.9% words/sec/thread:   30985 lr:  0.005414 avg.loss:  0.097291 ETA

Progress:  66.6% words/sec/thread:   29237 lr:  0.003337 avg.loss:  0.068932 ETA:   1h47m11s 56.9% words/sec/thread:   30446 lr:  0.004313 avg.loss:  0.079642 ETA:   2h13m 1s% words/sec/thread:   30445 lr:  0.004312 avg.loss:  0.079636 ETA:   2h13m 0s 56.9% words/sec/thread:   30440 lr:  0.004305 avg.loss:  0.079547 ETA:   2h12m49s 57.0% words/sec/thread:   30439 lr:  0.004305 avg.loss:  0.079542 ETA:   2h12m49s57.0% words/sec/thread:   30431 lr:  0.004297 avg.loss:  0.079444 ETA:   2h12m37s57.1% words/sec/thread:   30427 lr:  0.004295 avg.loss:  0.079411 ETA:   2h12m33s  30422 lr:  0.004293 avg.loss:  0.079387 ETA:   2h12m31s  30407 lr:  0.004286 avg.loss:  0.079299 ETA:   2h12m22s 57.2% words/sec/thread:   30403 lr:  0.004282 avg.loss:  0.079251 ETA:   2h12m16s 57.2% words/sec/thread:   30389 lr:  0.004277 avg.loss:  0.079179 ETA:   2h12m 9s 57.2% words/sec/thread:   30388 lr:  0.004276 avg.loss:  0.079173 ETA:   2h12m 9sm 7s 57.3% words/sec/thread:   30385 lr:  0.004274 avg.loss:  0

Progress:  77.9% words/sec/thread:   28341 lr:  0.002209 avg.loss:  0.059798 ETA:   1h13m11s 66.6% words/sec/thread:   29236 lr:  0.003335 avg.loss:  0.068913 ETA:   1h47m 8s 66.7% words/sec/thread:   29231 lr:  0.003327 avg.loss:  0.068832 ETA:   1h46m53s words/sec/thread:   29231 lr:  0.003321 avg.loss:  0.068772 ETA:   1h46m41s 66.8% words/sec/thread:   29230 lr:  0.003317 avg.loss:  0.068736 ETA:   1h46m34s 67.1% words/sec/thread:   29219 lr:  0.003291 avg.loss:  0.068492 ETA:   1h45m46s 67.2% words/sec/thread:   29214 lr:  0.003284 avg.loss:  0.068426 ETA:   1h45m33s 67.3% words/sec/thread:   29204 lr:  0.003275 avg.loss:  0.068342 ETA:   1h45m18s 67.3% words/sec/thread:   29201 lr:  0.003272 avg.loss:  0.068319 ETA:   1h45m14s 67.3% words/sec/thread:   29201 lr:  0.003270 avg.loss:  0.068292 ETA:   1h45m 8s 67.4% words/sec/thread:   29195 lr:  0.003261 avg.loss:  0.068216 ETA:   1h44m54sm52s lr:  0.003254 avg.loss:  0.068144 ETA:   1h44m40s 67.5% words/sec/thread:   29186 lr:  0.

Progress:  88.0% words/sec/thread:   27624 lr:  0.001203 avg.loss:  0.053625 ETA:   0h40m53s 77.9% words/sec/thread:   28338 lr:  0.002207 avg.loss:  0.059783 ETA:   1h13m 7s0.059658 ETA:   1h12m35s 78.2% words/sec/thread:   28291 lr:  0.002176 avg.loss:  0.059573 ETA:   1h12m14s4s 78.4% words/sec/thread:   28272 lr:  0.002161 avg.loss:  0.059468 ETA:   1h11m46s78.6% words/sec/thread:   28251 lr:  0.002142 avg.loss:  0.059334 ETA:   1h11m11s 78.6% words/sec/thread:   28250 lr:  0.002141 avg.loss:  0.059330 ETA:   1h11m10s  28249 lr:  0.002141 avg.loss:  0.059326 ETA:   1h11m 9s 0s 78.7% words/sec/thread:   28227 lr:  0.002125 avg.loss:  0.059221 ETA:   1h10m42s 78.8% words/sec/thread:   28223 lr:  0.002122 avg.loss:  0.059200 ETA:   1h10m36ss% words/sec/thread:   28221 lr:  0.002119 avg.loss:  0.059180 ETA:   1h10m31s57s% words/sec/thread:   28200 lr:  0.002096 avg.loss:  0.059025 ETA:   1h 9m47ss  28192 lr:  0.002084 avg.loss:  0.058944 ETA:   1h 9m24s 79.2% words/sec/thread:   28189 

Progress:  97.6% words/sec/thread:   26931 lr:  0.000239 avg.loss:  0.048870 ETA:   0h 8m19s 88.1% words/sec/thread:   27616 lr:  0.001190 avg.loss:  0.053557 ETA:   0h40m28s0h40m16s 9s 88.2% words/sec/thread:   27603 lr:  0.001178 avg.loss:  0.053486 ETA:   0h40m 3s38s 88.5% words/sec/thread:   27583 lr:  0.001154 avg.loss:  0.053357 ETA:   0h39m18s1s 88.6% words/sec/thread:   27570 lr:  0.001138 avg.loss:  0.053268 ETA:   0h38m45s 0.001138 avg.loss:  0.053268 ETA:   0h38m45s 88.6% words/sec/thread:   27570 lr:  0.001136 avg.loss:  0.053259 ETA:   0h38m42s 88.6% words/sec/thread:   27569 lr:  0.001135 avg.loss:  0.053254 ETA:   0h38m40s 88.7% words/sec/thread:   27563 lr:  0.001128 avg.loss:  0.053212 ETA:   0h38m24s0.053123 ETA:   0h37m52s 89.1% words/sec/thread:   27539 lr:  0.001090 avg.loss:  0.053010 ETA:   0h37m10s 89.2% words/sec/thread:   27530 lr:  0.001076 avg.loss:  0.052933 ETA:   0h36m41s 89.3% words/sec/thread:   27529 lr:  0.001074 avg.loss:  0.052921 ETA:   0h36m37s0h3

Progress: 100.0% words/sec/thread:   26767 lr:  0.000000 avg.loss:  0.047828 ETA:   0h 0m 0sh 8m19s97.6% words/sec/thread:   26931 lr:  0.000237 avg.loss:  0.048863 ETA:   0h 8m16s  26925 lr:  0.000222 avg.loss:  0.048796 ETA:   0h 7m45s 97.8% words/sec/thread:   26924 lr:  0.000217 avg.loss:  0.048774 ETA:   0h 7m34s 97.9% words/sec/thread:   26924 lr:  0.000215 avg.loss:  0.048762 ETA:   0h 7m29s% words/sec/thread:   26920 lr:  0.000203 avg.loss:  0.048711 ETA:   0h 7m 4s 98.0% words/sec/thread:   26904 lr:  0.000195 avg.loss:  0.048675 ETA:   0h 6m48s% words/sec/thread:   26901 lr:  0.000192 avg.loss:  0.048659 ETA:   0h 6m41s0h 6m30sh 6m25s 98.2% words/sec/thread:   26893 lr:  0.000179 avg.loss:  0.048602 ETA:   0h 6m15s 6m10s 98.3% words/sec/thread:   26892 lr:  0.000175 avg.loss:  0.048584 ETA:   0h 6m 6s40s 0.000151 avg.loss:  0.048480 ETA:   0h 5m16s 98.6% words/sec/thread:   26846 lr:  0.000142 avg.loss:  0.048441 ETA:   0h 4m58s 98.6% words/sec/thread:   26846 lr:  0.000142 a

In [68]:
# save model
model_sg.save_model("model_sg_fasttext_8.bin")

In [69]:
# load model
model_sg = fasttext.load_model("model_sg_fasttext_8.bin")

In [70]:
# relatedness model sg 8
print(relatedness(model_sg))

0.19100125783097885


In [71]:
# word analogy model sg 8
word_analogy(model_sg)

true_predicted 7 0.00035816618911174784
false_predicted 19537 0.9996418338108882
